<a href="https://colab.research.google.com/github/acabalAI/Dipromats_24_T2/blob/main/Dipromat_t2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1-Libraries and Dataset downloading

In [2]:
from google.colab import drive
import pandas as pd
import json

# This will prompt for authorization to access your Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path_t2_es = '/content/drive/My Drive/UNED/Dipromat/dipromats24_t2_train_es_v2.json'  # Adjust the path if your file is in a different directory
file_path_t2_en = '/content/drive/My Drive/UNED/Dipromat/dipromats24_t2_train_en_v2.json'


# Read the JSON file into a pandas DataFrame
df_t2_en = pd.read_json(file_path_t2_en)
df_t2_en = pd.json_normalize(df_t2_en['dataset'])
df_t2_es = pd.read_json(file_path_t2_es)
df_t2_es = pd.json_normalize(df_t2_es['dataset'])



In [4]:
df_t2_en['language'] = 'en'  # For the Spanish DataFrame
df_t2_es['language'] = 'es'  # For the English DataFrame

unified_df_t2 = pd.concat([df_t2_en, df_t2_es])

In [6]:
unified_df_t2.columns

Index(['id', 'country', 'tweet_id', 'text', 'CH1', 'CH2', 'CH3', 'CH4', 'CH5',
       'CH6', 'RU1', 'RU2', 'RU3', 'RU4', 'RU5', 'RU6', 'EU1', 'EU2', 'EU3',
       'EU4', 'EU5', 'EU6', 'US1', 'US2', 'US3', 'US4', 'US5', 'US6',
       'username', 'UTC', 'Tweet Type', 'rt&fav', 'language'],
      dtype='object')

In [ ]:
file_path = '/content/drive/My Drive/UNED/Dipromat/unified_df_t2.csv'
unified_df_t2.to_csv(file_path, index=False)


#2-Multiagent w/o semantic search

In [7]:
import pandas as pd

# Define the columns you want to check
columns_to_check = [
    'CH1', 'CH2', 'CH3', 'CH4', 'CH5', 'CH6',
    'RU1', 'RU2', 'RU3', 'RU4', 'RU5', 'RU6',
    'EU1', 'EU2', 'EU3', 'EU4', 'EU5', 'EU6',
    'US1', 'US2', 'US3', 'US4', 'US5', 'US6'
]

# Function to apply across each row
def categories_aggregator(row):
    return [col for col in columns_to_check if row[col] == 'yes']

# Create a new column 'yes_columns' which is a list of columns with 'yes'
df_t2_es['aggregated_values'] = df_t2_es.apply(categories_aggregator, axis=1)
df_t2_en['aggregated_values'] = df_t2_en.apply(categories_aggregator, axis=1)


In [10]:
!pip install transformers
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install OpenAI
!pip install langchain_community
import pandas as pd
from google.colab import drive
from transformers import BertTokenizer
PRETRAINED_LM = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_LM, do_lower_case=True)
from sklearn.model_selection import train_test_split
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
import pickle
from collections import Counter

from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union, Any, Optional, Type

import re
from langchain import PromptTemplate
from langchain.tools import BaseTool



from langchain.llms import OpenAI
from langchain.chains import LLMChain

import matplotlib.pyplot as plt
import seaborn as sns

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.4 MB/s eta 0:00:00


In [11]:
# Reference Examples and Keywords

class WrapperFrame_v1:
    def __init__(self, client):
        self.client = client

class CH_Agent(WrapperFrame_v1):
    def __init__(self, client):
        super().__init__(client)

        self.CH_agent = """
    Your primary role is to analyze a tweet {tweet} and categorize them according to predefined narrative themes that reflect different portrayals and perspectives of China.
    Your classification should help in understanding the overarching sentiments and strategic messaging in public discourse.
    Narratives to Detect:

    1:  The West as Immoral and Hostile:
        Identify tweets that depict Western countries, especially the US, as immoral, hostile, or decadent. Look for content where China is positioned as a victim of Western actions or policies.
        Example: It is the #US, not China that takes a predatory world view. Some in the #US have been seeking by all means to sabotage #ChinaUSrelations & deliberately disrupting China's development.
        Keywords: #US, predatory, sabotage, disrupt, reckless, immoral, hostile, decadent, confrontation, victim.
    2: China as a Benevolent Power:
        Classify tweets highlighting China's peaceful and cooperative international stance. Include tweets that mention China’s contributions to global peace, support for international law, and economic development in other nations.
        Example: #China pursues peaceful development. Apart from contributing to UN #Peacekeeping, it has not sent a single soldier to foreign land. It is not a threat but a builder of world peace, a contributor to global development & a defender of international order.
        Keywords: peaceful, development, #Peacekeeping, no soldier abroad, builder, world peace, global development, international order, cooperative, mutual benefits, justice, international law.
    3:  China’s Epic History:
        Detect tweets that discuss China's historical resilience and achievements, particularly those crediting the Chinese Communist Party with overcoming adversities and leading national modernization.
        Example: Una historia de ardua lucha por la salvación nacional primero y por la modernización nacional después, el propósito final siendo siempre el mismo: liderar al pueblo chino en la consecución de la dignidad humana
        Keywords: historical, resilience, ordeals, adversities, Chinese Communist Party, national dignity, epic, struggle, salvation, modernization.
    4:  China’s Political System and Values:
        Look for tweets advocating for socialism with Chinese characteristics and portraying it as aligned with the will of the Chinese people. Tweets should suggest that China’s political system supports genuine democracy and global peace.
        Example: China adheres to the path of peaceful development & will never seek hegemony or engage in expansion, a pledge that has never been made by #US. China is a force for good in the world, a force for global peace & prosperity. China does not want to threaten, challenge or replace anyone.
        Keywords: socialism, Chinese characteristics, political system, democracy, genuine, honorable, values, peace, no hegemony, expansion, global peace, prosperity.
    5: Success of the Chinese Communist Party’s Government:
        Identify tweets focusing on the government’s role in driving China’s technological, economic, and social advancements, such as achievements in 5G technology.
        Example: China leads the way on global standards for 5G and beyond. Congrats China, you have won the decisive battle against US in the technology war of the 21st century.
        Keywords: government, success, development, Chinese Communist Party, 5G, technology, economic forefront, minority coexistence, transformation, second largest economy.
    6:  China’s Cultural, Natural, and Heritage Appeal:
        Classify tweets that promote Chinese culture, traditions, natural beauty, or heritage sites. This includes mentions of historical cities, cultural festivities, and natural landscapes.
        Example: #PatrimoniodelaHumanidad. En la provincia de Shanxi se ubica Pingyao, la antigua ciudad China que aún conserva toda su historia y cultura en su arquitectura.  En este lugar relució la etnia Han.  ¿Ya la conocías?
        Keywords: culture, nature, heritage, traditions, natural beauty, historical, World Heritage Sites, diversity, cultural richness.
    Instructions for Classification:

    1- Read carefully the tweet {tweet}
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:

              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
              ).

        """
        self.prompt_CH_agent = PromptTemplate(template=self.CH_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_CH_agent = LLMChain(prompt=self.prompt_CH_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            CH_output = self.llm_chain_CH_agent.run({"tweet": self.tweet})
            return CH_output
        except Exception as e:
            print(e)
            return "Error China agent"



class RU_Agent(WrapperFrame_v1):
    def __init__(self, client):
        super().__init__(client)

        self.RU_agent = """

    You are to analyze and categorize a tweet {tweet} based on predefined Russian narrative themes.
    This classification is intended to help identify how Russia is portrayed and how it positions itself in global discourse through social media.

    Narratives to Detect:

    1: The West as Immoral and Hostile:
        Identify tweets that depict the West and its allies, like Ukraine and particularly the US, as immoral and hostile. These tweets often portray Russia as a victim of Western actions and criticize the liberal order as decadent.
        Example: We consider the situation with the rights of the #Russian-speaking population in #Latvia and #Estonia to be discriminatory. Statelessness is a shameful social phenomenon mainly applied to Russian-speaking residents, depriving them of basic democratic rights.
        Keywords: immoral, hostile, decadent, victim, discrimination, statelessness, #Latvia, #Estonia, rights, Russian-speaking.
    2: Russia Leads an Alternative System:
        Look for tweets that discuss Russia's leadership in a multilateral system that stands as an alternative to Western-sponsored systems. These tweets should emphasize Russia’s role in promoting a fairer global order.
        Example: #BRICS is a striking example of the effectiveness of multipolar diplomacy. Countries with different cultural & civilizational backgrounds have united on the basis of a constructive foreign policy philosophy, so in demand in the tumultuous modern world
        Keywords: alternative, multipolar, BRICS, cooperation, fairness, multilateral, non-Western, #BRICS, diplomacy.
    3:  Russia as a Benevolent Partner:
        Classify tweets that highlight Russia's role in fostering peace, solidarity, and development, portraying it as a reliable and just partner in international affairs.
        Example: Russia is connected with Azerbaijan and Armenia by shared history, culture, advanced humanitarian & economic ties. Our role as chief mediator in #NagornoKarabakh conflict is to help through the hot phase and find peaceful solutions to aggravated differences.
        Keywords: benevolent, partner, peace, solidarity, development, mediator, #NagornoKarabakh, humanitarian, economic ties, reliable.
    4: Russian History is Admirable:
        Detect tweets that showcase Russia's historical contributions to science, culture, the military, and economics, emphasizing its influential role in global history.
        Example: On 17 July 1942 the Battle of #Stalingrad - largest armed clash in history - commenced with a monstrous Nazi offensive aimed at crushing #RedArmy. Had it wavered, Nazism would have taken over the world. At #Stalingrad Soviet heroes won the battle & saved humanity
        Keywords: history, admirable, contributions, culture, military, economic, scientific, #Stalingrad, Soviet, heroes.
    5:  Russia at the World’s Forefront:
        Identify tweets that demonstrate Russia's contemporary successes in various domains such as economics, military, science, society, politics, and technology.
        Example: The chairman of Standing Committee on Vaccination Thomas Mertens, Germany: “It's a good vaccine that will probably be approved in EU at some point. The Russian researchers are very experienced with vaccinations. Sputnik V is cleverly built”.
        Keywords: forefront, successes, economic, military, scientific, societal, political, technological, vaccine, Sputnik
    6:  Russia's Cultural, Natural, and Heritage Appeal:
        Find tweets promoting Russian culture, traditions, natural beauty, and heritage sites, emphasizing the diversity and richness of Russian society.
        Example: From #Moscow to #SaintPetersburg, from #Kaliningrad to #Kamchatka, from the #Caucasus  to #Crimea - one country, myriads of places to see, wonders to behold, vistas to enjoy, people to meet.
        Keywords: interesting, culture, nature, heritage, traditions, natural beauty, #Moscow, #SaintPetersburg, #Kaliningrad, #Kamchatka, #Caucasus, #Crimea.

   Instructions for Classification:

    1- Read carefully the tweet {tweet}
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:
              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
             ).

        """
        self.prompt_RU_agent = PromptTemplate(template=self.RU_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_RU_agent = LLMChain(prompt=self.prompt_RU_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            RU_output = self.llm_chain_RU_agent.run({"tweet": self.tweet})
            return RU_output
        except Exception as e:
            print(e)
            return "Error Russia agent"

class EU_Agent(WrapperFrame_v1):
    def __init__(self, client):
        super().__init__(client)

        self.EU_agent = """

    Your role is to analyze and categorize a {tweet} according to predefined narrative themes related to the portrayal of the European Union.
    This will aid in understanding how the EU is represented in public discussions on social media.

    Narratives to Detect:

    1: The European Union is a Successful Story:
        Identify tweets that highlight the EU’s historical achievements, cooperation among member states, and learning from past mistakes, emphasizing its foundational values of peace, unity, and solidarity.
        Example: In 1950,R.Schuman made the emblematic proposals that founded the EU. Today we mark the 70th anniversary of this historic moment.This difficult period challenges us to reaffirm our values & keep sight of what Europe is & what it stands for:peace,unity & solidarity
        Keywords: successful, overcoming, cooperation, historical mistakes, values, peace, unity, solidarity, Schuman, anniversary, #EuropeanUnion.
    2: The European Union is United:
        Look for tweets that stress the EU's unity and solidarity, showcasing its ability to remain cohesive despite internal differences and challenges.
        Example: Only united and in a spirit of solidarity are we able to maintain our Union's strength. Countries, regions and citizens across the EU have extended a helping hand during the crisis.  Read stories of European solidarity.
        Keywords: united, agreements, solidarity, genuine, European sentiment, helping hand, crisis, strength, #EuropeanSolidarity.
    3:  The European Union is Useful for Its Citizens:
        Classify tweets that promote EU policies and their benefits, demonstrating how European measures tangibly improve the lives of its citizens.
        Example: Our new #MigrationEU pact sets a predictable and reliable system with: - more efficient & faster procedures - a fair sharing of responsibility & solidarity This approach will restore trust between EU countries and confidence in our capacity to manage migration.
        Keywords: useful, policies, improve lives, predictable, reliable, system, migration, responsibility, #MigrationEU.
    4: The European Union as an Avant-Garde Political Actor:
        Detect tweets that position the EU as a leader in tackling contemporary and future challenges such as climate change, digitalization, and regulation.
        Example: Europe is the leader in the fight against climate change. We decided to cut our greenhouse gas emissions of at least 55% by 2030.
        Keywords: avant-garde, pioneer, regulation, climate change, digitalization, emissions cut, climate neutral, #EUGreenDeal, #SOTEU.
    5: The European Union as a Global Champion of Fair Causes:
        Identify tweets that highlight the EU’s role in defending democracy, human rights, and supporting minority and gender rights, portraying the EU as a leader in promoting global justice and stability.
        Example: Respect for human rights is a pillar of free, stable, prosperous societies. The EU is proud to be at the forefront of its promotion and protection worldwide.
        Keywords: champion, democracy, human rights, gender rights, minority rights, promotion, protection, #HumanRights.
    6: The European Union Contributes to a Better World:
        Find tweets that illustrate the EU’s contributions to global development, peace, safety, and well-being, emphasizing its role as a key partner for neighboring and global societies.
        Example: #Africa is Europe’s closest neighbour. To strengthen our ties with our sister continent, the EU proposes five key partnerships. With the new #EUAfricaStrategy we can pursue common goals together.
        Keywords: contributes, development, peace, safety, well-being, global partner, #Africa, humanitarian, assistance, #EUAfricaStrategy.

    Instructions for Classification:

    1- Read carefully the tweet {tweet}.
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:
              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
              ).

        """
        self.prompt_EU_agent = PromptTemplate(template=self.EU_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_EU_agent = LLMChain(prompt=self.prompt_EU_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            EU_output = self.llm_chain_EU_agent.run({"tweet": self.tweet})
            return EU_output
        except Exception as e:
            print(e)
            return "Error EU agent"


class US_Agent(WrapperFrame_v1):
    def __init__(self, client):
        super().__init__(client)

        self.US_agent = """

Your task is to analyze and categorize a tweet {tweet} according to predefined narrative themes that reflect various portrayals of the United States.
This classification will contribute to understanding the representation of the U.S. in public discourse on social media.

Narratives to Detect:

    1: The US has an Honorable Tradition:
        Identify tweets that highlight the enduring legacy and foundational values of the U.S., emphasizing historical achievements and moments that define American identity.
        Example: Freedom of religion was the driving force behind some of the earliest defining moments of American identity - our Founding Fathers made it our No. 1 freedom for a reason.
        Keywords: honorable, tradition, values, history, Founding Fathers, freedom of religion, identity.
    2: The US has an Admirable Society:
        Look for tweets that promote the diversity and richness of American society, including its ethnic groups, civil society, business sectors, and military heroes.
        Example: Each February, the United States celebrates African-American History Month, also known as Black History Month. This annual observance recognizes the important achievements by African Americans in U.S. history.
        Keywords: admirable, society, ethnic groups, civil society, companies, military heroes, African-American History Month.
    3: The US has Hostile Enemies:
        Classify tweets that portray the U.S. as facing threats from immoral adversaries, often emphasizing these challenges to global peace and liberal values.
        Example: @khamenei_ir’s corrupt regime should listen to the people of #Iran. They want their culture and their country back.
        Keywords: hostile, enemies, immoral adversaries, global peace, liberal values, threat, @khamenei_ir, #Iran.
    4:  The US is a Force for Good:
        Detect tweets showcasing America’s efforts in supporting human rights, freedoms, oppressed peoples, and its contributions toward peace, development, and security.
        Example:America stands with the suffering people of Venezuela and Cuba and Nicaragua in their righteous struggle for liberty. We’ll continue working for the day when we will become the first fully free hemisphere in all of human history. We’ll have that day.
        Keywords: force for good, human rights, freedoms, oppressed peoples, peace, development, security, Venezuela, Cuba, Nicaragua.
    5: The US is a Great International Ally:
        Identify tweets that underscore the U.S.'s strong support for its allies, praising the alliances and collaborations that bolster global security and policy alignment.
        Example: Proud to support American policy and stand with our great ally Israel. New guidelines outlined today protect Israeli producers and put the United States, and the world, on track towards recognizing reality.
        Keywords: international ally, support, allies, American policy, Israel, shared values, reality.
    6:  The US is at the Forefront:
        Find tweets that position the U.S. as a leader in economy, technology, military, and trade, emphasizing innovation and competitive advantages on the global stage.
        Example: Glad to discuss ways to maintain America’s edge in innovation and technology with the university research community. Academic values such as openness, transparency, reciprocity, and merit-based competition make U.S. universities the best in the world.
        Keywords: forefront, leader, economy, technology, military, trade, innovation, environment, emissions.
    Instructions for Classification:

    1- Read carefully the tweet {tweet}
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:
              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
              ).

        """
        self.prompt_US_agent = PromptTemplate(template=self.US_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_US_agent = LLMChain(prompt=self.prompt_US_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            US_output = self.llm_chain_US_agent.run({"tweet": self.tweet})
            return US_output
        except Exception as e:
            print(e)
            return "Error US agent"



In [12]:
import json
import time

class EvaluationProcessPipeline:

    def __init__(self, client, row):
        self.client = client
        self.row = row
        self.country = self.row['country']
        self.text = self.row["text"]
        self.country_code = ''  # Placeholder for country code
        if self.country == 'USA':
            self.agent = US_Agent(self.client)
            self.country_code = 'US'
        elif self.country == 'EU':
            self.agent = EU_Agent(self.client)
            self.country_code = 'EU'
        elif self.country == 'Russia':
            self.agent = RU_Agent(self.client)
            self.country_code = 'RU'
        elif self.country == 'China':
            self.agent = CH_Agent(self.client)
            self.country_code = 'CH'
        else:
            # Raise an error if the country does not match any known option
            raise ValueError(f"No valid agent found for the specified country: {self.country}")

    def process_tweet(self):
        max_attempts = 5  # Define maximum number of retry attempts
        attempts = 0

        while attempts < max_attempts:
            try:
                # Run the selected Agent
                entry_result = self.agent._run_agent(self.text)
                print("Raw entry_result:", entry_result)  # Debugging line to see what is being returned

                # Attempt to parse the JSON data
                data = json.loads(entry_result)
                label_numbers = data["classification"]
                # Transform the list of numbers into a list of formatted strings with the country code
                labels = [f"{self.country_code}{num}" for num in label_numbers]
                print("Extracted labels:", labels)  # Debugging line to confirm label extraction
                return labels  # Successful parsing and return, exit loop
            except (json.JSONDecodeError, Exception) as error:
                # Handle both JSON parsing errors and other exceptions
                print(f"Error encountered (attempt {attempts + 1}):", error)
                attempts += 1
                time.sleep(1)  # Sleep before retrying

        print("Failed after", max_attempts, "attempts.")
        return []



In [ ]:
import os
os.environ["OPENAI_API_KEY"] = openai_key
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

#2.1- English Dataset

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer
import time

In [ ]:
predicted_values_en=[]
real_values_en=[]
for i,entry in enumerate (df_t2_en):
  agent = EvaluationProcessPipeline(llm, df_t2_en.iloc[i])
  label = agent.process_tweet()
  predicted_values_en.append(label)
  real_values_en.append(df_t2_en.iloc[i]['aggregated_values'])
  time.sleep(10)




In [ ]:
#Reference Example

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score

# Example data

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=['CH1', 'CH2', 'CH3', 'CH4', 'CH5', 'CH6', 'RU1', 'RU2', 'RU3', 'RU4', 'RU5', 'RU6', 'EU1', 'EU2', 'EU3', 'EU4', 'EU5', 'EU6', 'US1', 'US2', 'US3', 'US4', 'US5', 'US6'])

# Binarize the data
y_true = mlb.fit_transform(real_values_en)

y_pred = mlb.transform(predicted_values_en)

# Calculate F1 Score
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print("F1 Score (Macro):", f1_macro)
print("F1 Score (Micro):", f1_micro)
print("F1 Score (Weighted):", f1_weighted)



In [ ]:
report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0)

print(report)

              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00         4
         CH2       0.33      1.00      0.50         3
         CH3       0.50      1.00      0.67         3
         CH4       1.00      0.50      0.67         4
         CH5       1.00      0.50      0.67         2
         CH6       1.00      1.00      1.00         2
         RU1       1.00      1.00      1.00         2
         RU2       0.67      1.00      0.80         2
         RU3       1.00      0.50      0.67         2
         RU4       1.00      1.00      1.00         1
         RU5       1.00      1.00      1.00         1
         RU6       1.00      1.00      1.00         1
         EU1       0.50      1.00      0.67         1
         EU2       0.50      1.00      0.67         1
         EU3       1.00      1.00      1.00         1
         EU4       1.00      1.00      1.00         1
         EU5       0.50      1.00      0.67         1
         EU6       1.00    

In [ ]:
#Example reference + Keywords

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score

# Example data

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=['CH1', 'CH2', 'CH3', 'CH4', 'CH5', 'CH6', 'RU1', 'RU2', 'RU3', 'RU4', 'RU5', 'RU6', 'EU1', 'EU2', 'EU3', 'EU4', 'EU5', 'EU6', 'US1', 'US2', 'US3', 'US4', 'US5', 'US6'])

# Binarize the data
y_true = mlb.fit_transform(real_values_en)

y_pred = mlb.transform(predicted_values_en)

# Calculate F1 Score
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print("F1 Score (Macro):", f1_macro)
print("F1 Score (Micro):", f1_micro)
print("F1 Score (Weighted):", f1_weighted)



F1 Score (Macro): 0.8909722222222222
F1 Score (Micro): 0.8275862068965517
F1 Score (Weighted): 0.855128205128205


In [ ]:
report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0)

print(report)

              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00         4
         CH2       0.33      1.00      0.50         3
         CH3       0.60      1.00      0.75         3
         CH4       1.00      0.50      0.67         4
         CH5       1.00      1.00      1.00         2
         CH6       1.00      1.00      1.00         2
         RU1       1.00      1.00      1.00         2
         RU2       0.67      1.00      0.80         2
         RU3       1.00      0.50      0.67         2
         RU4       1.00      1.00      1.00         1
         RU5       1.00      1.00      1.00         1
         RU6       1.00      1.00      1.00         1
         EU1       0.50      1.00      0.67         1
         EU2       1.00      1.00      1.00         1
         EU3       1.00      1.00      1.00         1
         EU4       1.00      1.00      1.00         1
         EU5       1.00      1.00      1.00         1
         EU6       1.00    

#2.2- Spanish Tweets

In [ ]:
class WrapperFrame_v1_es:
    def __init__(self, client):
        self.client = client

class CH_Agent(WrapperFrame_v1_es):
    def __init__(self, client):
        super().__init__(client)

        self.CH_agent = """
    Your primary role is to analyze a tweet {tweet} and categorize them according to predefined narrative themes that reflect different portrayals and perspectives of China.
    Your classification should help in understanding the overarching sentiments and strategic messaging in public discourse.
    Narratives to Detect:

    1:  The West as Immoral and Hostile:
        Identify tweets that depict Western countries, especially the US, as immoral, hostile, or decadent. Look for content where China is positioned as a victim of Western actions or policies.
        Example: "Difamación y calumnias, intervención en los asuntos ajenos, sanciones unilaterales, provocación militar, la ineficiencia contra covid19, doble rasero de derechos humanos y aislacionismo constituyen los siete pecados capitales estadounidenses contra China."
        Keywords: Difamación, intervención, sanciones unilaterales, provocación militar, ineficiencia, doble rasero, aislacionismo.
    2: China as a Benevolent Power:
        Classify tweets highlighting China's peaceful and cooperative international stance. Include tweets that mention China’s contributions to global peace, support for international law, and economic development in other nations.
        Example: "Al ofrecer asistencia al exterior, no tenemos ningún objetivo geopolítico, ni cálculo económico, ni condición política alguna. En vez de ser El Salvador, #China quiere ser auxiliador benéfico y sincero socio que siempre rema acompasado con amigos en momentos difíciles."
        Keywords: Asistencia, beneficio, cooperativo, socio, ayuda, sin condiciones.
    3:  China’s Epic History:
        Detect tweets that discuss China's historical resilience and achievements, particularly those crediting the Chinese Communist Party with overcoming adversities and leading national modernization.
        Example: "Una historia de ardua lucha por la salvación nacional primero y por la modernización nacional después, el propósito final siendo siempre el mismo: liderar al pueblo chino en la consecución de la dignidad humana."
        Keywords: Historia, lucha, salvación nacional, modernización, liderazgo, dignidad humana.
    4:  China’s Political System and Values:
        Look for tweets advocating for socialism with Chinese characteristics and portraying it as aligned with the will of the Chinese people. Tweets should suggest that China’s political system supports genuine democracy and global peace.
        Example: "Si la democracia es 1 medicina, la forma de administrarla ha de ser diversa en función de cada caso diferente. La china, 1 democracia popular que combina votos indirectos con amplia consulta política, bajo el liderazgo del #PCCh, que gobierna en coalición con otros 8 partidos, funciona."
        Keywords: Democracia, administración diversa, consulta política, liderazgo del PCCh, coalición.
    5: Success of the Chinese Communist Party’s Government:
        Identify tweets focusing on the government’s role in driving China’s technological, economic, and social advancements, such as achievements in 5G technology.
        Example: "Nuevo hito de científicos chinos: un equipo de la Universidad de Ciencia y Tecnología de #China logró establecer una red de comunicación cuántica segura y estable en 4600 kms. Esperan su uso comercial en un futuro cercano."
        Keywords: Avance científico, comunicación cuántica, tecnología, éxito, innovación.
    6:  China’s Cultural, Natural, and Heritage Appeal:
        Classify tweets that promote Chinese culture, traditions, natural beauty, or heritage sites. This includes mentions of historical cities, cultural festivities, and natural landscapes.
        Example: "#PatrimoniodelaHumanidad. En la provincia de Shanxi se ubica Pingyao, la antigua ciudad China que aún conserva toda su historia y cultura en su arquitectura. En este lugar relució la etnia Han. ¿Ya la conocías?"
        Keywords: Patrimonio de la Humanidad, cultura, historia, naturaleza, herencia cultural, Pingyao.

    Instructions for Classification:

    1- Read carefully the tweet {tweet}
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:

              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
              ).

        """
        self.prompt_CH_agent = PromptTemplate(template=self.CH_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_CH_agent = LLMChain(prompt=self.prompt_CH_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            CH_output = self.llm_chain_CH_agent.run({"tweet": self.tweet})
            return CH_output
        except Exception as e:
            print(e)
            return "Error China agent"



class RU_Agent(WrapperFrame_v1_es):
    def __init__(self, client):
        super().__init__(client)

        self.RU_agent = """

    You are to analyze and categorize a tweet {tweet} in Spanish based on predefined Russian narrative themes.
    This classification is intended to help identify how Russia is portrayed and how it positions itself in global discourse through social media.

    Narratives to Detect:

    1: The West as Immoral and Hostile:
        Identify tweets that depict the West and its allies, like Ukraine and particularly the US, as immoral and hostile. These tweets often portray Russia as a victim of Western actions and criticize the liberal order as decadent.
        Example: "S.#Lavrov: Lamentablemente a nuestros socios estadounidenses les importa no la cooperación con un país en particular, sino que el Gobierno de este país sea leal a #EEUU. De ahí las políticas agresivas contra #Venezuela #Cuba #Nicaragua."
        Keywords: inmoral, hostil, decadente, lealtad, políticas agresivas, EE.UU., sanciones.
    2: Russia Leads an Alternative System:
        Look for tweets that discuss Russia's leadership in a multilateral system that stands as an alternative to Western-sponsored systems. These tweets should emphasize Russia’s role in promoting a fairer global order.
        Example: "#Lavrov: La voz solidaria de los países #BRICS en apoyo del orden global democrático y multipolar, que va formándose y se basa en el respeto mutuo de intereses legítimos de todos los Estados y todas las normas y principios de la Carta de la ONU, es pertinente hoy más que nunca."
        Keywords: alternativo, multipolar, BRICS, democrático, respeto mutuo, orden global.

    3:  Russia as a Benevolent Partner:
        Classify tweets that highlight Russia's role in fostering peace, solidarity, and development, portraying it as a reliable and just partner in international affairs.
        Example: "#Zajárova: Coordinando su política exterior y actuando de mancomún, nuestros países contribuyen a promover una agenda constructiva en los principales foros multilaterales."
        Keywords: benevolente, solidaridad, cooperación, paz, desarrollo, mediador.

    4: Russian History is Admirable:
        Detect tweets that showcase Russia's historical contributions to science, culture, the military, and economics, emphasizing its influential role in global history.
        Example: "#UnDíaComoHoy, en 1844 nació destacado pintor Ilia #Repin. Entre sus obras más emblemáticas son: Iván el Terrible y su hijo, Procesión de Pascua en la región de Kursk, Cosacos zaporogos escribiendo una carta al Sultán."
        Keywords: historia, admirable, cultura, contribuciones, Repin, arte.

    5:  Russia at the World’s Forefront:
        Identify tweets that demonstrate Russia's contemporary successes in various domains such as economics, military, science, society, politics, and technology.
        Example: "El centro nacional de investigación virológica y biotecnológica #Vektor ha patentado segunda vacuna para #Covid19 - #EpiVakCorona. A diferencia de #SputnikV, esta vacuna contiene fragmentos del virus y da lugar a una reacción inmunológica."
        Keywords: liderazgo, innovación, científico, tecnológico, vacuna, vanguardia.
    6:  Russia's Cultural, Natural, and Heritage Appeal:
        Find tweets promoting Russian culture, traditions, natural beauty, and heritage sites, emphasizing the diversity and richness of Russian society.
        Example: "Rusia es un país con historia milenaria y herencia única con extensos territorios & diversas culturas. Más de 190 grupos étnicos llaman Rusia su casa. Somos distintos pero somos uno. Feliz #DiaDeRusia !"
        Keywords: cultura, naturaleza, herencia, diversidad, etnicidad, celebración.

   Instructions for Classification:

    1- Read carefully the tweet {tweet}
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:
              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
             ).

        """
        self.prompt_RU_agent = PromptTemplate(template=self.RU_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_RU_agent = LLMChain(prompt=self.prompt_RU_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            RU_output = self.llm_chain_RU_agent.run({"tweet": self.tweet})
            return RU_output
        except Exception as e:
            print(e)
            return "Error Russia agent"

class EU_Agent(WrapperFrame_v1_es):
    def __init__(self, client):
        super().__init__(client)

        self.EU_agent = """

    Your role is to analyze and categorize a {tweet} in Spanish according to predefined narrative themes related to the portrayal of the European Union.
    This will aid in understanding how the EU is represented in public discussions on social media.

    Narratives to Detect:

    1: The European Union is a Successful Story:
        Identify tweets that highlight the EU’s historical achievements, cooperation among member states, and learning from past mistakes, emphasizing its foundational values of peace, unity, and solidarity.
        Example: "Hoy se cumplen 25 años de la masacre de #Srebrenica, una de las páginas más oscuras de Europa y un fracaso colectivo de proteger a los que necesitaban protección. No olvidemos la historia para que no se repita."
        Keywords: superación, cooperación, historia, reconciliación, aprendizaje, protección.
    2: The European Union is United:
        Look for tweets that stress the EU's unity and solidarity, showcasing its ability to remain cohesive despite internal differences and challenges.
        Example: "La historia muestra que una Europa fuerte en la escena internacional empieza con nuestra unidad interna. Esta unidad nos permitirá también proyectar la fortaleza de Europa en el mundo. Solo unidos podremos afrontar los retos globales de hoy en día #EuropeDay #DiaDeEuropa."
        Keywords: unidad, solidaridad, fuerza, cooperación, retos globales.
    3:  The European Union is Useful for Its Citizens:
        Classify tweets that promote EU policies and their benefits, demonstrating how European measures tangibly improve the lives of its citizens.
        Example: "@vonderleyen - COVID19 ha cambiado nuestras vidas y traído tragedias. Pero ahora hay esperanza. La UE ha invertido en la investigación y el desarrollo de vacunas. Hemos asegurado dosis para toda nuestra población. La vacunación comenzará pronto. Una historia de éxito europeo."
        Keywords: útil, mejoramiento, políticas, ciudadanos, investigación, desarrollo, vacunas.
    4: The European Union as an Avant-Garde Political Actor:
        Detect tweets that position the EU as a leader in tackling contemporary and future challenges such as climate change, digitalization, and regulation.
        Example: "#DíaMundialDelMedioAmbiente 🌍🌿 Europa trabaja por un futuro verde, respetuoso con el medio ambiente, las personas y el clima. Y lo hará con: 🌿Pacto Verde Europeo #EUGreenDeal 🌿Plan de acción para la #CircularEconomy 🌿Estrategia de Biodiversidad para 2030 #EUBiodiversity."
        Keywords: avant-garde, innovador, líder, cambio climático, digitalización, Pacto Verde, economía circular.
    5: The European Union as a Global Champion of Fair Causes:
        Identify tweets that highlight the EU’s role in defending democracy, human rights, and supporting minority and gender rights, portraying the EU as a leader in promoting global justice and stability.
        Example: "El Parlamento Europeo declara la Unión Europea «zona de libertad para las personas LGBTIQ». La igualdad de género se sitúa en el núcleo de la acción exterior de la UE. El Plan de Acción UE en Género busca promover la igualdad y garantizar que mujeres y niñas de todo el mundo participen plenamente en la construcción de la democracia, la justicia, la paz y la seguridad."
        Keywords: defensa, democracia, derechos humanos, igualdad de género, minorías, justicia.
    6: The European Union Contributes to a Better World:
        Find tweets that illustrate the EU’s contributions to global development, peace, safety, and well-being, emphasizing its role as a key partner for neighboring and global societies.
        Example: "La #UE está asegurando más de 15,000 millones de euros para ayudar a nuestros socios en todo el mundo a combatir el #coronavirus. Queremos asegurarnos de que la lucha tenga éxito en todo el mundo."
        Keywords: contribución, desarrollo, paz, seguridad, bienestar global, cooperación internacional.
    Instructions for Classification:

    1- Read carefully the tweet {tweet}.
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:
              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
              ).

        """
        self.prompt_EU_agent = PromptTemplate(template=self.EU_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_EU_agent = LLMChain(prompt=self.prompt_EU_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            EU_output = self.llm_chain_EU_agent.run({"tweet": self.tweet})
            return EU_output
        except Exception as e:
            print(e)
            return "Error EU agent"


class US_Agent(WrapperFrame_v1_es):
    def __init__(self, client):
        super().__init__(client)

        self.US_agent = """

Your task is to analyze and categorize a tweet {tweet} in Spanish according to predefined narrative themes that reflect various portrayals of the United States.
This classification will contribute to understanding the representation of the U.S. in public discourse on social media.

Narratives to Detect:

    1: The US has an Honorable Tradition:
        Identify tweets that highlight the enduring legacy and foundational values of the U.S., emphasizing historical achievements and moments that define American identity.
        Example: "La libertad de expresión, fundamental para una sociedad libre y la comunicación abierta ayudan a impulsar ideas para promover la justicia y reducir la desigualdad. EE.UU. valora el derecho de todas las personas a expresarse libremente."
        Keywords: libertad de expresión, sociedad libre, justicia, valores.

    2: The US has an Admirable Society:
        Look for tweets that promote the diversity and richness of American society, including its ethnic groups, civil society, business sectors, and military heroes.
        Example: "Cada año en los Estados Unidos, del 15 de septiembre al 15 de octubre, celebramos la riqueza de las tradiciones y la cultura de la #HerenciaHispana. En @laembajada de los #EEUU en Asunción, nuestra comunidad refleja el orgullo y el poder de esta gran herencia."
        Keywords: diversidad, Herencia Hispana, tradiciones, cultura.

    3: The US has Hostile Enemies:
        Classify tweets that portray the U.S. as facing threats from immoral adversaries, often emphasizing these challenges to global peace and liberal values.
        Example: "El régimen iraní ha perseguido a la comunidad religiosa bahaí desde la Revolución Islámica de 1979. EEUU pide al régimen de Irán que ponga fin a la persecución de la comunidad religiosa bahaí."
        Keywords: enemigos, régimen iraní, persecución, defensa de los derechos.

    4:  The US is a Force for Good:
        Detect tweets showcasing America’s efforts in supporting human rights, freedoms, oppressed peoples, and its contributions toward peace, development, and security.
        Example: "Los Estados Unidos están con el pueblo cubano en su lucha por lograr su visión de una Cuba libre y más justa. El día en que su sueño de libertad se convierta en realidad lleva décadas de retraso, pero ese día llegará."
        Keywords: apoyo, libertad, derechos humanos, Cuba, justicia.
    5: The US is a Great International Ally:
        Identify tweets that underscore the U.S.'s strong support for its allies, praising the alliances and collaborations that bolster global security and policy alignment.
        Example: "Durante +70 años, el apoyo de EEUU a la @NATO ha sido inquebrantable. El vínculo Europa-EEUU ha convertido a la OTAN en la alianza más poderosa de la historia. EEUU mantiene su compromiso blindado con los aliados OTAN, nuestra seguridad y valores compartidos."
        Keywords: aliados, OTAN, seguridad, compromiso, valores compartidos.
    6:  The US is at the Forefront:
        Find tweets that position the U.S. as a leader in economy, technology, military, and trade, emphasizing innovation and competitive advantages on the global stage.
        Example: "Rumbo a Marte! Hoy fue el lanzamiento del #perseverancerover que estará explorando el planeta rojo. Como estadounidense, estoy orgullosa de lo que @Nasa ha logrado a través de la historia y de su continuo liderazgo en la exploración espacial."
        Keywords: liderazgo, innovación, NASA, exploración espacial, tecnología.
    Instructions for Classification:

    1- Read carefully the tweet {tweet}
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:
              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
              ).

        """
        self.prompt_US_agent = PromptTemplate(template=self.US_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_US_agent = LLMChain(prompt=self.prompt_US_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            US_output = self.llm_chain_US_agent.run({"tweet": self.tweet})
            return US_output
        except Exception as e:
            print(e)
            return "Error US agent"



In [ ]:
import json
import time

class EvaluationProcessPipeline:

    def __init__(self, client, row):
        self.client = client
        self.row = row
        self.country = self.row['country']
        self.text = self.row["text"]
        self.country_code = ''  # Placeholder for country code
        if self.country == 'USA':
            self.agent = US_Agent(self.client)
            self.country_code = 'US'
        elif self.country == 'EU':
            self.agent = EU_Agent(self.client)
            self.country_code = 'EU'
        elif self.country == 'Russia':
            self.agent = RU_Agent(self.client)
            self.country_code = 'RU'
        elif self.country == 'China':
            self.agent = CH_Agent(self.client)
            self.country_code = 'CH'
        else:
            # Raise an error if the country does not match any known option
            raise ValueError(f"No valid agent found for the specified country: {self.country}")

    def process_tweet(self):
        max_attempts = 5  # Define maximum number of retry attempts
        attempts = 0

        while attempts < max_attempts:
            try:
                # Run the selected Agent
                entry_result = self.agent._run_agent(self.text)
                print("Raw entry_result:", entry_result)  # Debugging line to see what is being returned

                # Attempt to parse the JSON data
                data = json.loads(entry_result)
                label_numbers = data["classification"]
                # Transform the list of numbers into a list of formatted strings with the country code
                labels = [f"{self.country_code}{num}" for num in label_numbers]
                print("Extracted labels:", labels)  # Debugging line to confirm label extraction
                return labels  # Successful parsing and return, exit loop
            except (json.JSONDecodeError, Exception) as error:
                # Handle both JSON parsing errors and other exceptions
                print(f"Error encountered (attempt {attempts + 1}):", error)
                attempts += 1
                time.sleep(1)  # Sleep before retrying

        print("Failed after", max_attempts, "attempts.")
        return []



In [ ]:
predicted_values_es=[]
real_values_es=[]
real_values_leaning_es=[]
for i,entry in enumerate (df_t2_es):
  agent = EvaluationProcessPipeline(llm, df_t2_es.iloc[i])
  label = agent.process_tweet()
  predicted_values_es.append(label)
  real_values_es.append(df_t2_es.iloc[i]['aggregated_values'])
  time.sleep(10)




In [ ]:
# Example and Keywords
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score

# Example data

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=['CH1', 'CH2', 'CH3', 'CH4', 'CH5', 'CH6', 'RU1', 'RU2', 'RU3', 'RU4', 'RU5', 'RU6', 'EU1', 'EU2', 'EU3', 'EU4', 'EU5', 'EU6', 'US1', 'US2', 'US3', 'US4', 'US5', 'US6'])

# Binarize the data
y_true = mlb.fit_transform(real_values_es)
y_pred = mlb.transform(predicted_values_es)

# Calculate F1 Score
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print("F1 Score (Macro):", f1_macro)
print("F1 Score (Micro):", f1_micro)
print("F1 Score (Weighted):", f1_weighted)



F1 Score (Macro): 0.5724206349206349
F1 Score (Micro): 0.7674418604651163
F1 Score (Weighted): 0.8095238095238095


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [ ]:
report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0)

print(report)

              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00         2
         CH2       0.67      1.00      0.80         2
         CH3       0.40      1.00      0.57         2
         CH4       0.67      1.00      0.80         2
         CH5       1.00      1.00      1.00         2
         CH6       1.00      0.50      0.67         4
         RU1       1.00      1.00      1.00         2
         RU2       0.50      1.00      0.67         2
         RU3       0.67      1.00      0.80         2
         RU4       0.50      1.00      0.67         2
         RU5       0.50      0.50      0.50         2
         RU6       1.00      1.00      1.00         2
         EU1       0.00      0.00      0.00         0
         EU2       0.67      1.00      0.80         2
         EU3       0.50      1.00      0.67         1
         EU4       1.00      1.00      1.00         1
         EU5       0.67      1.00      0.80         2
         EU6       1.00    

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score

# Example data

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=['CH1', 'CH2', 'CH3', 'CH4', 'CH5', 'CH6', 'RU1', 'RU2', 'RU3', 'RU4', 'RU5', 'RU6', 'EU1', 'EU2', 'EU3', 'EU4', 'EU5', 'EU6', 'US1', 'US2', 'US3', 'US4', 'US5', 'US6'])

# Binarize the data
y_true = mlb.fit_transform(real_values_es)
y_pred = mlb.transform(predicted_values_es)

# Calculate F1 Score
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print("F1 Score (Macro):", f1_macro)
print("F1 Score (Micro):", f1_micro)
print("F1 Score (Weighted):", f1_weighted)



F1 Score (Macro): 0.4759920634920635
F1 Score (Micro): 0.674698795180723
F1 Score (Weighted): 0.6961640211640211


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [ ]:
report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0)

print(report)

              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00         2
         CH2       0.67      1.00      0.80         2
         CH3       0.33      1.00      0.50         2
         CH4       0.67      1.00      0.80         2
         CH5       0.67      1.00      0.80         2
         CH6       1.00      0.75      0.86         4
         RU1       0.00      0.00      0.00         2
         RU2       0.33      0.50      0.40         2
         RU3       0.50      0.50      0.50         2
         RU4       0.50      0.50      0.50         2
         RU5       1.00      0.50      0.67         2
         RU6       0.50      0.50      0.50         2
         EU1       0.00      0.00      0.00         0
         EU2       0.67      1.00      0.80         2
         EU3       0.33      1.00      0.50         1
         EU4       1.00      1.00      1.00         1
         EU5       0.67      1.00      0.80         2
         EU6       1.00    

#3-Multiagent w semantic search

In [ ]:
pip install transformers torch

In [ ]:
import pandas as pd
from transformers import BertModel, BertTokenizer
import torch


In [ ]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import DataLoader, Dataset

# Initialize tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
model = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
if torch.cuda.is_available():
    model = model.cuda()



tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts.iloc[idx]  # Just return the text  # Remov

In [ ]:
def collate_batch(batch):
    # Tokenize the batch of texts
    tokenized_batch = tokenizer(batch, padding=True, truncation=True, max_length=512, return_tensors='pt')
    return tokenized_batch

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class TextDataset(Dataset):
    def __init__(self, texts):
        if isinstance(texts, list):
            self.texts = pd.DataFrame(texts, columns=['text'])
        elif isinstance(texts, pd.DataFrame):
            self.texts = texts
        else:
            raise TypeError("Input texts must be either a list of texts or a pandas DataFrame")

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        if isinstance(self.texts, pd.DataFrame):
            return self.texts.iloc[idx]['text']
        return self.texts[idx]


def get_multilingual_embeddings(texts, batch_size=32):
    dataset = TextDataset(texts)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)
    embeddings = []

    for batch in data_loader:
        # Assume `batch` is now a properly collated batch of tensors, including 'input_ids' and 'attention_mask'
        input_ids = batch['input_ids'].to('cuda' if torch.cuda.is_available() else 'cpu')
        attention_mask = batch['attention_mask'].to('cuda' if torch.cuda.is_available() else 'cpu')

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1).detach().cpu().numpy()
        embeddings.extend(batch_embeddings)

    return embeddings


In [ ]:
tweet=unified_df_t1.iloc[0]['text']

In [ ]:
embeddings = get_multilingual_embeddings(unified_df_t2, batch_size=16)

unified_df_t2['text_embeddings']= embeddings

In [ ]:
class WrapperFrame_v2:
    def __init__(self, client):
        self.client = client

class CH_Agent(WrapperFrame_v2):
    def __init__(self, client):
        super().__init__(client)

        self.CH_agent = """
    Your primary role is to analyze a tweet {tweet} and categorize them according to predefined narrative themes that reflect different portrayals and perspectives of China.
    For that purpouse you will have close example {example} as reference.
    Your classification should help in understanding the overarching sentiments and strategic messaging in public discourse.
    Narratives to Detect:

    1:  The West as Immoral and Hostile:
        Identify tweets that depict Western countries, especially the US, as immoral, hostile, or decadent. Look for content where China is positioned as a victim of Western actions or policies.

    2: China as a Benevolent Power:
        Classify tweets highlighting China's peaceful and cooperative international stance. Include tweets that mention China’s contributions to global peace, support for international law, and economic development in other nations.

    3:  China’s Epic History:
        Detect tweets that discuss China's historical resilience and achievements, particularly those crediting the Chinese Communist Party with overcoming adversities and leading national modernization.

    4:  China’s Political System and Values:
        Look for tweets advocating for socialism with Chinese characteristics and portraying it as aligned with the will of the Chinese people. Tweets should suggest that China’s political system supports genuine democracy and global peace.

    5: Success of the Chinese Communist Party’s Government:
        Identify tweets focusing on the government’s role in driving China’s technological, economic, and social advancements, such as achievements in 5G technology.

    6:  China’s Cultural, Natural, and Heritage Appeal:
        Classify tweets that promote Chinese culture, traditions, natural beauty, or heritage sites. This includes mentions of historical cities, cultural festivities, and natural landscapes.

    Instructions for Classification:

    1- Read carefully the tweet {tweet} and the reference example {example}.
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:

              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
              ).

        """
        self.prompt_CH_agent = PromptTemplate(template=self.CH_agent, input_variables=["tweet","example"])

        # Corrected the LLMChain instantiation
        self.llm_chain_CH_agent = LLMChain(prompt=self.prompt_CH_agent, llm=self.client)

    def _run_agent(self,tweet,example):
        self.tweet = tweet
        self.example=example

        try:
            CH_output = self.llm_chain_CH_agent.run({"tweet": self.tweet,"example":self.example})
            return CH_output
        except Exception as e:
            print(e)
            return "Error China agent"



class RU_Agent(WrapperFrame_v2):
    def __init__(self, client):
        super().__init__(client)

        self.RU_agent = """

    You are to analyze and categorize a tweet {tweet} based on predefined Russian narrative themes.
    For that purpouse you will have close example {example} as reference.
    This classification is intended to help identify how Russia is portrayed and how it positions itself in global discourse through social media.

    Narratives to Detect:

    1: The West as Immoral and Hostile:
        Identify tweets that depict the West and its allies, like Ukraine and particularly the US, as immoral and hostile. These tweets often portray Russia as a victim of Western actions and criticize the liberal order as decadent.

    2: Russia Leads an Alternative System:
        Look for tweets that discuss Russia's leadership in a multilateral system that stands as an alternative to Western-sponsored systems. These tweets should emphasize Russia’s role in promoting a fairer global order.

    3:  Russia as a Benevolent Partner:
        Classify tweets that highlight Russia's role in fostering peace, solidarity, and development, portraying it as a reliable and just partner in international affairs.

    4: Russian History is Admirable:
        Detect tweets that showcase Russia's historical contributions to science, culture, the military, and economics, emphasizing its influential role in global history.

    5:  Russia at the World’s Forefront:
        Identify tweets that demonstrate Russia's contemporary successes in various domains such as economics, military, science, society, politics, and technology.

    6:  Russia's Cultural, Natural, and Heritage Appeal:
        Find tweets promoting Russian culture, traditions, natural beauty, and heritage sites, emphasizing the diversity and richness of Russian society.

   Instructions for Classification:

    1- Read carefully the tweet {tweet} and the reference example {example}.
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:
              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
             ).

        """
        self.prompt_RU_agent = PromptTemplate(template=self.RU_agent, input_variables=["tweet","example"])

        # Corrected the LLMChain instantiation
        self.llm_chain_RU_agent = LLMChain(prompt=self.prompt_RU_agent, llm=self.client)

    def _run_agent(self,tweet,example):
        self.tweet = tweet
        self.example=example

        try:
            RU_output = self.llm_chain_RU_agent.run({"tweet": self.tweet,"example":self.example})
            return RU_output
        except Exception as e:
            print(e)
            return "Error Russia agent"

class EU_Agent(WrapperFrame_v2):
    def __init__(self, client):
        super().__init__(client)

        self.EU_agent = """

    Your role is to analyze and categorize a {tweet} according to predefined narrative themes related to the portrayal of the European Union.
    For that purpouse you will have close example {example} as reference.
    This will aid in understanding how the EU is represented in public discussions on social media.

    Narratives to Detect:

    1: The European Union is a Successful Story:
        Identify tweets that highlight the EU’s historical achievements, cooperation among member states, and learning from past mistakes, emphasizing its foundational values of peace, unity, and solidarity.

    2: The European Union is United:
        Look for tweets that stress the EU's unity and solidarity, showcasing its ability to remain cohesive despite internal differences and challenges.

    3:  The European Union is Useful for Its Citizens:
        Classify tweets that promote EU policies and their benefits, demonstrating how European measures tangibly improve the lives of its citizens.

    4: The European Union as an Avant-Garde Political Actor:
        Detect tweets that position the EU as a leader in tackling contemporary and future challenges such as climate change, digitalization, and regulation.

    5: The European Union as a Global Champion of Fair Causes:
        Identify tweets that highlight the EU’s role in defending democracy, human rights, and supporting minority and gender rights, portraying the EU as a leader in promoting global justice and stability.

    6: The European Union Contributes to a Better World:
        Find tweets that illustrate the EU’s contributions to global development, peace, safety, and well-being, emphasizing its role as a key partner for neighboring and global societies.

    Instructions for Classification:

    1- Read carefully the tweet {tweet} and the reference example {example}.
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:
              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
              ).

        """
        self.prompt_EU_agent = PromptTemplate(template=self.EU_agent, input_variables=["tweet","example"])

        # Corrected the LLMChain instantiation
        self.llm_chain_EU_agent = LLMChain(prompt=self.prompt_EU_agent, llm=self.client)

    def _run_agent(self,tweet,example):
        self.tweet = tweet
        self.example=example

        try:
            EU_output = self.llm_chain_EU_agent.run({"tweet": self.tweet,"example":self.example})
            return EU_output
        except Exception as e:
            print(e)
            return "Error EU agent"


class US_Agent(WrapperFrame_v2):
    def __init__(self, client):
        super().__init__(client)

        self.US_agent = """

Your task is to analyze and categorize a tweet {tweet} according to predefined narrative themes that reflect various portrayals of the United States.
For that purpouse you will have close example {example} as reference.
This classification will contribute to understanding the representation of the U.S. in public discourse on social media.

Narratives to Detect:

    1: The US has an Honorable Tradition:
        Identify tweets that highlight the enduring legacy and foundational values of the U.S., emphasizing historical achievements and moments that define American identity.

    2: The US has an Admirable Society:
        Look for tweets that promote the diversity and richness of American society, including its ethnic groups, civil society, business sectors, and military heroes.

    3: The US has Hostile Enemies:
        Classify tweets that portray the U.S. as facing threats from immoral adversaries, often emphasizing these challenges to global peace and liberal values.

    4:  The US is a Force for Good:
        Detect tweets showcasing America’s efforts in supporting human rights, freedoms, oppressed peoples, and its contributions toward peace, development, and security.

    5: The US is a Great International Ally:
        Identify tweets that underscore the U.S.'s strong support for its allies, praising the alliances and collaborations that bolster global security and policy alignment.

    6:  The US is at the Forefront:
        Find tweets that position the U.S. as a leader in economy, technology, military, and trade, emphasizing innovation and competitive advantages on the global stage.

    Instructions for Classification:

    1- Read carefully the tweet {tweet} and the reference example {example}.
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:
              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
              ).

        """
        self.prompt_US_agent = PromptTemplate(template=self.US_agent, input_variables=["tweet","example"])

        # Corrected the LLMChain instantiation
        self.llm_chain_US_agent = LLMChain(prompt=self.prompt_US_agent, llm=self.client)

    def _run_agent(self,tweet,example):
        self.tweet = tweet
        self.example=example

        try:
            US_output = self.llm_chain_US_agent.run({"tweet": self.tweet,"example":self.example})
            return US_output
        except Exception as e:
            print(e)
            return "Error US agent"


In [ ]:
import json
from scipy.spatial.distance import cosine
import random


def clean_text(text):
    print(text)
    """Remove newlines and extra spaces from the text."""
    return " ".join(text.replace('\n', ' ').split())
def similarity_search(entry, train_dataset):
    # Adjusted to use indexing for pandas Series
    tweet = entry['text']
    tweet = str(tweet) # Adjust the key to match your DataFrame's column name for tweets
    country = entry['country']  # Adjust the key to match your DataFrame's column name for country

    # Clean the tweet
    tweet = clean_text(tweet)

    # Detect language
    language = entry['language']
    print(f"Detected Language: {language}")

    # Get embeddings
    tweet_embedding = get_multilingual_embeddings([tweet])[0]  # Assuming function returns an array of embeddings
    print("Embedding generated.")

    # Prepare to gather similar tweets
    similar_tweets = []

    # Filter DataFrame by language and country
    subset = train_dataset[
        (train_dataset['language'] == language) &
        (train_dataset['country'] == country)]

    # Check if the subset is empty
    if subset.empty:
        print("No matching entries found.")
        return []

    # Compute cosine similarity
    similarities = [1 - cosine(tweet_embedding, np.array(emb)) for emb in subset['text_embeddings']]
    # Get top 3 indices
    top_indices = np.argsort(similarities)[-1:]
    # Append each similar tweet and its label to the list
    for idx in top_indices:
        similar_tweets.append({
            "text": subset.iloc[idx]['text'],
            "classification": subset.iloc[idx]['aggregated_values']
        })
    random.shuffle(similar_tweets)

    return similar_tweets


In [ ]:
embeddings = get_multilingual_embeddings(unified_df_t2, batch_size=1)

unified_df_t2['text_embeddings']= embeddings

In [ ]:
def remove_country_code(values):
    return [value[2:] for value in values]

# Apply the function to the DataFrame
unified_df_t2['aggregated_values'] = unified_df_t2['aggregated_values'].apply(remove_country_code)

unified_df_t2.head()

,id,country,tweet_id,text,CH1,CH2,CH3,CH4,CH5,CH6,...,US3,US4,US5,US6,username,UTC,Tweet Type,rt&fav,language,aggregated_values
0,0,China,1282957959601872898,"It is the #US, not China that takes a predator...",yes,yes,no,no,no,no,...,no,no,no,no,spokespersonchn,2020-07-14 08:39:48+00:00,Tweet,3230.0,en,"[1, 2]"
1,1,China,1334887444559384577,#China pursues peaceful development. Apart fro...,no,yes,no,no,no,no,...,no,no,no,no,ambliuxiaoming,2020-12-04 15:49:02+00:00,quoted,131.0,en,[2]
2,2,China,1271775491767271426,The Chinese nation has experienced many ordeal...,no,leaning,yes,no,no,no,...,no,no,no,no,li_xiaosi,2020-06-13 12:04:40+00:00,Tweet,230.0,en,[3]
3,3,China,1264091015557808128,Socialism with Chinese Characteristics is choi...,no,yes,no,yes,no,no,...,no,no,no,no,ambchenxiaodong,2020-05-23 07:09:18+00:00,Tweet,42.0,en,"[2, 4]"
4,4,China,1287053000968556544,China leads the way on global standards for 5G...,no,no,no,no,yes,no,...,no,no,no,no,beltandroaddesk,2020-07-25 15:52:02+00:00,Tweet,659.0,en,[5]


In [ ]:
import json

class EvaluationProcessPipeline_v2:

    def __init__(self, client, row,reference_df):
        self.client = client
        self.row = row
        self.country = self.row['country']
        self.text = row["text"]
        self.example=similarity_search(row, reference_df)
        print(self.example)

        self.country_code = ''  # Placeholder for country code
        if self.country == 'USA':
            self.agent = US_Agent(self.client)
            self.country_code = 'US'
        elif self.country == 'EU':
            self.agent = EU_Agent(self.client)
            self.country_code = 'EU'
        elif self.country == 'Russia':
            self.agent = RU_Agent(self.client)
            self.country_code = 'RU'
        elif self.country == 'China':
            self.agent = CH_Agent(self.client)
            self.country_code = 'CH'
        else:
            # Raise an error if the country does not match any known option
            raise ValueError(f"No valid agent found for the specified country: {self.country}")

    def process_tweet(self):
        max_attempts = 5  # Define maximum number of retry attempts
        attempts = 0

        while attempts < max_attempts:
            try:
                # Run the selected Agent
                entry_result = self.agent._run_agent(self.text, self.example)
                print("Raw entry_result:", entry_result)  # Debugging line to see what is being returned

                # Attempt to parse the JSON data
                data = json.loads(entry_result)
                label_numbers = data["classification"]
                # Transform the list of numbers into a list of formatted strings with the country code
                labels = [f"{self.country_code}{num}" for num in label_numbers]
                print("Extracted labels:", labels)  # Debugging line to confirm label extraction
                return labels  # Successful parsing and return, exit loop
            except (json.JSONDecodeError, Exception) as error:
                # Handle both JSON parsing errors and other exceptions
                print(f"Error encountered (attempt {attempts + 1}):", error)
                attempts += 1
                time.sleep(1)  # Sleep before retrying

        print("Failed after", max_attempts, "attempts.")
        return []





In [ ]:
predicted_values_en = []
real_values_en = []

# Iterate over each row in the DataFrame
for i in range(len(unified_df_t2)):
    # Select the current row as the test set
    X_test = unified_df_t2.iloc[i:i+1]

    # Select the rest of the DataFrame as the reference set, excluding the current test row
    X_ref = pd.concat([unified_df_t2.iloc[:i], unified_df_t2.iloc[i+1:]])

    # Instantiate and use the evaluation pipeline
    agent = EvaluationProcessPipeline_v2(llm, X_test.iloc[0], X_ref)
    label = agent.process_tweet()

    predicted_values_en.append(label)
    real_values_en.append(unified_df_t2.iloc[i]['aggregated_values'])
    #real_values_leaning_en.append(unified_df_t2.iloc[i]['aggregated_values_leaning'])

    # Delay for 10 seconds to avoid rate limiting or similar issues
    time.sleep(10)


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score

# Example data

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=['1', '2', '3', '4', '5', '6'])

# Binarize the data
y_true = mlb.fit_transform(real_values_en)
y_pred = mlb.transform(numeric_values)

# Calculate F1 Score
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print("F1 Score (Macro):", f1_macro)
print("F1 Score (Micro):", f1_micro)
print("F1 Score (Weighted):", f1_weighted)



F1 Score (Macro): 0.7148468815135481
F1 Score (Micro): 0.7058823529411765
F1 Score (Weighted): 0.7124676955185429


In [ ]:
report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0)

print(report)

              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00         2
         CH2       0.67      1.00      0.80         2
         CH3       0.33      1.00      0.50         2
         CH4       0.67      1.00      0.80         2
         CH5       0.67      1.00      0.80         2
         CH6       1.00      0.75      0.86         4
         RU1       0.00      0.00      0.00         2
         RU2       0.33      0.50      0.40         2
         RU3       0.50      0.50      0.50         2
         RU4       0.50      0.50      0.50         2
         RU5       1.00      0.50      0.67         2
         RU6       0.50      0.50      0.50         2
         EU1       0.00      0.00      0.00         0
         EU2       0.67      1.00      0.80         2
         EU3       0.33      1.00      0.50         1
         EU4       1.00      1.00      1.00         1
         EU5       0.67      1.00      0.80         2
         EU6       1.00    

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score

# Example data

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=['CH1', 'CH2', 'CH3', 'CH4', 'CH5', 'CH6', 'RU1', 'RU2', 'RU3', 'RU4', 'RU5', 'RU6', 'EU1', 'EU2', 'EU3', 'EU4', 'EU5', 'EU6', 'US1', 'US2', 'US3', 'US4', 'US5', 'US6'])

# Binarize the data
y_true = mlb.fit_transform(real_values_en)
y_true_leaning = mlb.fit_transform(real_values_leaning_en)
y_pred = mlb.transform(predicted_values_en)

# Calculate F1 Score
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print("F1 Score (Macro):", f1_macro)
print("F1 Score (Micro):", f1_micro)
print("F1 Score (Weighted):", f1_weighted)





F1 Score (Macro): 0.6809323165940814
F1 Score (Micro): 0.6738351254480287
F1 Score (Weighted): 0.6902026887451145


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score

# Example data

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=['CH1', 'CH2', 'CH3', 'CH4', 'CH5', 'CH6', 'RU1', 'RU2', 'RU3', 'RU4', 'RU5', 'RU6', 'EU1', 'EU2', 'EU3', 'EU4', 'EU5', 'EU6', 'US1', 'US2', 'US3', 'US4', 'US5', 'US6'])

# Binarize the data
y_true = mlb.fit_transform(real_values_en[:51])
y_true_leaning = mlb.fit_transform(real_values_leaning_en[:51])
y_pred = mlb.transform(predicted_values_en[:51])

# Calculate F1 Score
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print("F1 Score (Macro):", f1_macro)
print("F1 Score (Micro):", f1_micro)
print("F1 Score (Weighted):", f1_weighted)





F1 Score (Macro): 0.6925354737854738
F1 Score (Micro): 0.6758620689655173
F1 Score (Weighted): 0.6925184043828111


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score

# Example data

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=['CH1', 'CH2', 'CH3', 'CH4', 'CH5', 'CH6', 'RU1', 'RU2', 'RU3', 'RU4', 'RU5', 'RU6', 'EU1', 'EU2', 'EU3', 'EU4', 'EU5', 'EU6', 'US1', 'US2', 'US3', 'US4', 'US5', 'US6'])

# Binarize the data
y_true = mlb.fit_transform(real_values_en[51:])
y_true_leaning = mlb.fit_transform(real_values_leaning_en[51:])
y_pred = mlb.transform(predicted_values_en[51:])

# Calculate F1 Score
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print("F1 Score (Macro):", f1_macro)
print("F1 Score (Micro):", f1_micro)
print("F1 Score (Weighted):", f1_weighted)





F1 Score (Macro): 0.6707010582010583
F1 Score (Micro): 0.6716417910447761
F1 Score (Weighted): 0.6907113462669018


#4-Inference Test Dataset

In [ ]:

file_path_t2_es = '/content/drive/My Drive/UNED/Dipromat/dipromats24_t2_test_es.json'  # Adjust the path if your file is in a different directory
file_path_t2_en = '/content/drive/My Drive/UNED/Dipromat/dipromats24_t2_test_en.json'



df_t2_en = pd.read_json(file_path_t2_en)
df_t2_en = pd.json_normalize(df_t2_en['dataset'])
df_t2_es = pd.read_json(file_path_t2_es)
df_t2_es = pd.json_normalize(df_t2_es['dataset'])



#4.1 English

In [ ]:
# Reference Examples and Keywords

class WrapperFrame_v1:
    def __init__(self, client):
        self.client = client

class CH_Agent(WrapperFrame_v1):
    def __init__(self, client):
        super().__init__(client)

        self.CH_agent = """
    Your primary role is to analyze a tweet {tweet} and categorize them according to predefined narrative themes that reflect different portrayals and perspectives of China.
    Your classification should help in understanding the overarching sentiments and strategic messaging in public discourse.
    Narratives to Detect:

    1:  The West as Immoral and Hostile:
        Identify tweets that depict Western countries, especially the US, as immoral, hostile, or decadent. Look for content where China is positioned as a victim of Western actions or policies.
        Example: It is the #US, not China that takes a predatory world view. Some in the #US have been seeking by all means to sabotage #ChinaUSrelations & deliberately disrupting China's development.
        Keywords: #US, predatory, sabotage, disrupt, reckless, immoral, hostile, decadent, confrontation, victim.
    2: China as a Benevolent Power:
        Classify tweets highlighting China's peaceful and cooperative international stance. Include tweets that mention China’s contributions to global peace, support for international law, and economic development in other nations.
        Example: #China pursues peaceful development. Apart from contributing to UN #Peacekeeping, it has not sent a single soldier to foreign land. It is not a threat but a builder of world peace, a contributor to global development & a defender of international order.
        Keywords: peaceful, development, #Peacekeeping, no soldier abroad, builder, world peace, global development, international order, cooperative, mutual benefits, justice, international law.
    3:  China’s Epic History:
        Detect tweets that discuss China's historical resilience and achievements, particularly those crediting the Chinese Communist Party with overcoming adversities and leading national modernization.
        Example: Una historia de ardua lucha por la salvación nacional primero y por la modernización nacional después, el propósito final siendo siempre el mismo: liderar al pueblo chino en la consecución de la dignidad humana
        Keywords: historical, resilience, ordeals, adversities, Chinese Communist Party, national dignity, epic, struggle, salvation, modernization.
    4:  China’s Political System and Values:
        Look for tweets advocating for socialism with Chinese characteristics and portraying it as aligned with the will of the Chinese people. Tweets should suggest that China’s political system supports genuine democracy and global peace.
        Example: China adheres to the path of peaceful development & will never seek hegemony or engage in expansion, a pledge that has never been made by #US. China is a force for good in the world, a force for global peace & prosperity. China does not want to threaten, challenge or replace anyone.
        Keywords: socialism, Chinese characteristics, political system, democracy, genuine, honorable, values, peace, no hegemony, expansion, global peace, prosperity.
    5: Success of the Chinese Communist Party’s Government:
        Identify tweets focusing on the government’s role in driving China’s technological, economic, and social advancements, such as achievements in 5G technology.
        Example: China leads the way on global standards for 5G and beyond. Congrats China, you have won the decisive battle against US in the technology war of the 21st century.
        Keywords: government, success, development, Chinese Communist Party, 5G, technology, economic forefront, minority coexistence, transformation, second largest economy.
    6:  China’s Cultural, Natural, and Heritage Appeal:
        Classify tweets that promote Chinese culture, traditions, natural beauty, or heritage sites. This includes mentions of historical cities, cultural festivities, and natural landscapes.
        Example: #PatrimoniodelaHumanidad. En la provincia de Shanxi se ubica Pingyao, la antigua ciudad China que aún conserva toda su historia y cultura en su arquitectura.  En este lugar relució la etnia Han.  ¿Ya la conocías?
        Keywords: culture, nature, heritage, traditions, natural beauty, historical, World Heritage Sites, diversity, cultural richness.
    Instructions for Classification:

    1- Read carefully the tweet {tweet}
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:

              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
              ).

        """
        self.prompt_CH_agent = PromptTemplate(template=self.CH_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_CH_agent = LLMChain(prompt=self.prompt_CH_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            CH_output = self.llm_chain_CH_agent.run({"tweet": self.tweet})
            return CH_output
        except Exception as e:
            print(e)
            return "Error China agent"



class RU_Agent(WrapperFrame_v1):
    def __init__(self, client):
        super().__init__(client)

        self.RU_agent = """

    You are to analyze and categorize a tweet {tweet} based on predefined Russian narrative themes.
    This classification is intended to help identify how Russia is portrayed and how it positions itself in global discourse through social media.

    Narratives to Detect:

    1: The West as Immoral and Hostile:
        Identify tweets that depict the West and its allies, like Ukraine and particularly the US, as immoral and hostile. These tweets often portray Russia as a victim of Western actions and criticize the liberal order as decadent.
        Example: We consider the situation with the rights of the #Russian-speaking population in #Latvia and #Estonia to be discriminatory. Statelessness is a shameful social phenomenon mainly applied to Russian-speaking residents, depriving them of basic democratic rights.
        Keywords: immoral, hostile, decadent, victim, discrimination, statelessness, #Latvia, #Estonia, rights, Russian-speaking.
    2: Russia Leads an Alternative System:
        Look for tweets that discuss Russia's leadership in a multilateral system that stands as an alternative to Western-sponsored systems. These tweets should emphasize Russia’s role in promoting a fairer global order.
        Example: #BRICS is a striking example of the effectiveness of multipolar diplomacy. Countries with different cultural & civilizational backgrounds have united on the basis of a constructive foreign policy philosophy, so in demand in the tumultuous modern world
        Keywords: alternative, multipolar, BRICS, cooperation, fairness, multilateral, non-Western, #BRICS, diplomacy.
    3:  Russia as a Benevolent Partner:
        Classify tweets that highlight Russia's role in fostering peace, solidarity, and development, portraying it as a reliable and just partner in international affairs.
        Example: Russia is connected with Azerbaijan and Armenia by shared history, culture, advanced humanitarian & economic ties. Our role as chief mediator in #NagornoKarabakh conflict is to help through the hot phase and find peaceful solutions to aggravated differences.
        Keywords: benevolent, partner, peace, solidarity, development, mediator, #NagornoKarabakh, humanitarian, economic ties, reliable.
    4: Russian History is Admirable:
        Detect tweets that showcase Russia's historical contributions to science, culture, the military, and economics, emphasizing its influential role in global history.
        Example: On 17 July 1942 the Battle of #Stalingrad - largest armed clash in history - commenced with a monstrous Nazi offensive aimed at crushing #RedArmy. Had it wavered, Nazism would have taken over the world. At #Stalingrad Soviet heroes won the battle & saved humanity
        Keywords: history, admirable, contributions, culture, military, economic, scientific, #Stalingrad, Soviet, heroes.
    5:  Russia at the World’s Forefront:
        Identify tweets that demonstrate Russia's contemporary successes in various domains such as economics, military, science, society, politics, and technology.
        Example: The chairman of Standing Committee on Vaccination Thomas Mertens, Germany: “It's a good vaccine that will probably be approved in EU at some point. The Russian researchers are very experienced with vaccinations. Sputnik V is cleverly built”.
        Keywords: forefront, successes, economic, military, scientific, societal, political, technological, vaccine, Sputnik
    6:  Russia's Cultural, Natural, and Heritage Appeal:
        Find tweets promoting Russian culture, traditions, natural beauty, and heritage sites, emphasizing the diversity and richness of Russian society.
        Example: From #Moscow to #SaintPetersburg, from #Kaliningrad to #Kamchatka, from the #Caucasus  to #Crimea - one country, myriads of places to see, wonders to behold, vistas to enjoy, people to meet.
        Keywords: interesting, culture, nature, heritage, traditions, natural beauty, #Moscow, #SaintPetersburg, #Kaliningrad, #Kamchatka, #Caucasus, #Crimea.

   Instructions for Classification:

    1- Read carefully the tweet {tweet}
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:
              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
             ).

        """
        self.prompt_RU_agent = PromptTemplate(template=self.RU_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_RU_agent = LLMChain(prompt=self.prompt_RU_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            RU_output = self.llm_chain_RU_agent.run({"tweet": self.tweet})
            return RU_output
        except Exception as e:
            print(e)
            return "Error Russia agent"

class EU_Agent(WrapperFrame_v1):
    def __init__(self, client):
        super().__init__(client)

        self.EU_agent = """

    Your role is to analyze and categorize a {tweet} according to predefined narrative themes related to the portrayal of the European Union.
    This will aid in understanding how the EU is represented in public discussions on social media.

    Narratives to Detect:

    1: The European Union is a Successful Story:
        Identify tweets that highlight the EU’s historical achievements, cooperation among member states, and learning from past mistakes, emphasizing its foundational values of peace, unity, and solidarity.
        Example: In 1950,R.Schuman made the emblematic proposals that founded the EU. Today we mark the 70th anniversary of this historic moment.This difficult period challenges us to reaffirm our values & keep sight of what Europe is & what it stands for:peace,unity & solidarity
        Keywords: successful, overcoming, cooperation, historical mistakes, values, peace, unity, solidarity, Schuman, anniversary, #EuropeanUnion.
    2: The European Union is United:
        Look for tweets that stress the EU's unity and solidarity, showcasing its ability to remain cohesive despite internal differences and challenges.
        Example: Only united and in a spirit of solidarity are we able to maintain our Union's strength. Countries, regions and citizens across the EU have extended a helping hand during the crisis.  Read stories of European solidarity.
        Keywords: united, agreements, solidarity, genuine, European sentiment, helping hand, crisis, strength, #EuropeanSolidarity.
    3:  The European Union is Useful for Its Citizens:
        Classify tweets that promote EU policies and their benefits, demonstrating how European measures tangibly improve the lives of its citizens.
        Example: Our new #MigrationEU pact sets a predictable and reliable system with: - more efficient & faster procedures - a fair sharing of responsibility & solidarity This approach will restore trust between EU countries and confidence in our capacity to manage migration.
        Keywords: useful, policies, improve lives, predictable, reliable, system, migration, responsibility, #MigrationEU.
    4: The European Union as an Avant-Garde Political Actor:
        Detect tweets that position the EU as a leader in tackling contemporary and future challenges such as climate change, digitalization, and regulation.
        Example: Europe is the leader in the fight against climate change. We decided to cut our greenhouse gas emissions of at least 55% by 2030.
        Keywords: avant-garde, pioneer, regulation, climate change, digitalization, emissions cut, climate neutral, #EUGreenDeal, #SOTEU.
    5: The European Union as a Global Champion of Fair Causes:
        Identify tweets that highlight the EU’s role in defending democracy, human rights, and supporting minority and gender rights, portraying the EU as a leader in promoting global justice and stability.
        Example: Respect for human rights is a pillar of free, stable, prosperous societies. The EU is proud to be at the forefront of its promotion and protection worldwide.
        Keywords: champion, democracy, human rights, gender rights, minority rights, promotion, protection, #HumanRights.
    6: The European Union Contributes to a Better World:
        Find tweets that illustrate the EU’s contributions to global development, peace, safety, and well-being, emphasizing its role as a key partner for neighboring and global societies.
        Example: #Africa is Europe’s closest neighbour. To strengthen our ties with our sister continent, the EU proposes five key partnerships. With the new #EUAfricaStrategy we can pursue common goals together.
        Keywords: contributes, development, peace, safety, well-being, global partner, #Africa, humanitarian, assistance, #EUAfricaStrategy.

    Instructions for Classification:

    1- Read carefully the tweet {tweet}.
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:
              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
              ).

        """
        self.prompt_EU_agent = PromptTemplate(template=self.EU_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_EU_agent = LLMChain(prompt=self.prompt_EU_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            EU_output = self.llm_chain_EU_agent.run({"tweet": self.tweet})
            return EU_output
        except Exception as e:
            print(e)
            return "Error EU agent"


class US_Agent(WrapperFrame_v1):
    def __init__(self, client):
        super().__init__(client)

        self.US_agent = """

Your task is to analyze and categorize a tweet {tweet} according to predefined narrative themes that reflect various portrayals of the United States.
This classification will contribute to understanding the representation of the U.S. in public discourse on social media.

Narratives to Detect:

    1: The US has an Honorable Tradition:
        Identify tweets that highlight the enduring legacy and foundational values of the U.S., emphasizing historical achievements and moments that define American identity.
        Example: Freedom of religion was the driving force behind some of the earliest defining moments of American identity - our Founding Fathers made it our No. 1 freedom for a reason.
        Keywords: honorable, tradition, values, history, Founding Fathers, freedom of religion, identity.
    2: The US has an Admirable Society:
        Look for tweets that promote the diversity and richness of American society, including its ethnic groups, civil society, business sectors, and military heroes.
        Example: Each February, the United States celebrates African-American History Month, also known as Black History Month. This annual observance recognizes the important achievements by African Americans in U.S. history.
        Keywords: admirable, society, ethnic groups, civil society, companies, military heroes, African-American History Month.
    3: The US has Hostile Enemies:
        Classify tweets that portray the U.S. as facing threats from immoral adversaries, often emphasizing these challenges to global peace and liberal values.
        Example: @khamenei_ir’s corrupt regime should listen to the people of #Iran. They want their culture and their country back.
        Keywords: hostile, enemies, immoral adversaries, global peace, liberal values, threat, @khamenei_ir, #Iran.
    4:  The US is a Force for Good:
        Detect tweets showcasing America’s efforts in supporting human rights, freedoms, oppressed peoples, and its contributions toward peace, development, and security.
        Example:America stands with the suffering people of Venezuela and Cuba and Nicaragua in their righteous struggle for liberty. We’ll continue working for the day when we will become the first fully free hemisphere in all of human history. We’ll have that day.
        Keywords: force for good, human rights, freedoms, oppressed peoples, peace, development, security, Venezuela, Cuba, Nicaragua.
    5: The US is a Great International Ally:
        Identify tweets that underscore the U.S.'s strong support for its allies, praising the alliances and collaborations that bolster global security and policy alignment.
        Example: Proud to support American policy and stand with our great ally Israel. New guidelines outlined today protect Israeli producers and put the United States, and the world, on track towards recognizing reality.
        Keywords: international ally, support, allies, American policy, Israel, shared values, reality.
    6:  The US is at the Forefront:
        Find tweets that position the U.S. as a leader in economy, technology, military, and trade, emphasizing innovation and competitive advantages on the global stage.
        Example: Glad to discuss ways to maintain America’s edge in innovation and technology with the university research community. Academic values such as openness, transparency, reciprocity, and merit-based competition make U.S. universities the best in the world.
        Keywords: forefront, leader, economy, technology, military, trade, innovation, environment, emissions.
    Instructions for Classification:

    1- Read carefully the tweet {tweet}
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:
              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
              ).

        """
        self.prompt_US_agent = PromptTemplate(template=self.US_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_US_agent = LLMChain(prompt=self.prompt_US_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            US_output = self.llm_chain_US_agent.run({"tweet": self.tweet})
            return US_output
        except Exception as e:
            print(e)
            return "Error US agent"



In [ ]:
import json
import time

class EvaluationProcessPipeline:

    def __init__(self, client, row):
        self.client = client
        self.row = row
        self.country = self.row['country']
        self.text = self.row["text"]
        self.country_code = ''  # Placeholder for country code
        if self.country == 'USA':
            self.agent = US_Agent(self.client)
            self.country_code = 'US'
        elif self.country == 'European Union':
            self.agent = EU_Agent(self.client)
            self.country_code = 'EU'
        elif self.country == 'Russia':
            self.agent = RU_Agent(self.client)
            self.country_code = 'RU'
        elif self.country == 'China':
            self.agent = CH_Agent(self.client)
            self.country_code = 'CH'
        else:
            # Raise an error if the country does not match any known option
            raise ValueError(f"No valid agent found for the specified country: {self.country}")

    def process_tweet(self):
        max_attempts = 5  # Define maximum number of retry attempts
        attempts = 0

        while attempts < max_attempts:
            try:
                # Run the selected Agent
                entry_result = self.agent._run_agent(self.text)
                print("Raw entry_result:", entry_result)  # Debugging line to see what is being returned

                # Attempt to parse the JSON data
                data = json.loads(entry_result)
                label_numbers = data["classification"]
                # Transform the list of numbers into a list of formatted strings with the country code
                labels = [f"{self.country_code}{num}" for num in label_numbers]
                print("Extracted labels:", labels)  # Debugging line to confirm label extraction
                return labels  # Successful parsing and return, exit loop
            except (json.JSONDecodeError, Exception) as error:
                # Handle both JSON parsing errors and other exceptions
                print(f"Error encountered (attempt {attempts + 1}):", error)
                attempts += 1
                time.sleep(1)  # Sleep before retrying

        print("Failed after", max_attempts, "attempts.")
        return []



In [ ]:
len(df_t2_en)

800

In [ ]:
df_t2_en['narratives'] = None

for i in range(len(df_t2_en)):
    print(i)
    # Extract the row as a Series
    entry = df_t2_en.iloc[i]

    # Process the tweet through your custom function/class
    agent = EvaluationProcessPipeline(llm, entry)  # Adjust according to what 'llm' and 'entry' should be based on your class definition
    label = agent.process_tweet()

    # Assign the processed label to the 'narratives' column at the correct index
    df_t2_en.at[i, 'narratives'] = label

    # Pause for 10 seconds before processing the next entry
    time.sleep(10)

# Optional: Print the DataFrame to verify the updates
print(df_t2_en)




In [ ]:
df_t2_response_en = df_t2_en[['id', 'country', 'tweet_id', 'narratives']]

In [ ]:
# Convert the DataFrame to a JSON string
result_json = {
    "responses": df_t2_response_en.to_dict(orient='records')
}

import json
response_t2_en = json.dumps(result_json, indent=4)  # Serialize the dictionary to a JSON formatted string with indentation for readability

# Print the JSON output
print(response_t2_en)


{
    "responses": [
        {
            "language": "en",
            "id": 26,
            "country": "USA",
            "tweet_id": "1213081403014955008",
            "narratives": []
        },
        {
            "language": "en",
            "id": 78,
            "country": "Russia",
            "tweet_id": "1214456508496388098",
            "narratives": [
                "RU4"
            ]
        },
        {
            "language": "en",
            "id": 81,
            "country": "China",
            "tweet_id": "1214507512864444416",
            "narratives": []
        },
        {
            "language": "en",
            "id": 86,
            "country": "European Union",
            "tweet_id": "1214561065134018567",
            "narratives": [
                "EU2",
                "EU6"
            ]
        },
        {
            "language": "en",
            "id": 108,
            "country": "European Union",
            "tweet_id": "1214854305699782656",
   

In [ ]:
df_t2_en.to_json('/content/drive/My Drive/UNED/Dipromat/df_t2_en.json')


#4.2 Spanish

In [ ]:
class WrapperFrame_v1_es:
    def __init__(self, client):
        self.client = client

class CH_Agent(WrapperFrame_v1_es):
    def __init__(self, client):
        super().__init__(client)

        self.CH_agent = """
    Your primary role is to analyze a tweet {tweet} and categorize them according to predefined narrative themes that reflect different portrayals and perspectives of China.
    Your classification should help in understanding the overarching sentiments and strategic messaging in public discourse.
    Narratives to Detect:

    1:  The West as Immoral and Hostile:
        Identify tweets that depict Western countries, especially the US, as immoral, hostile, or decadent. Look for content where China is positioned as a victim of Western actions or policies.
        Example: "Difamación y calumnias, intervención en los asuntos ajenos, sanciones unilaterales, provocación militar, la ineficiencia contra covid19, doble rasero de derechos humanos y aislacionismo constituyen los siete pecados capitales estadounidenses contra China."
        Keywords: Difamación, intervención, sanciones unilaterales, provocación militar, ineficiencia, doble rasero, aislacionismo.
    2: China as a Benevolent Power:
        Classify tweets highlighting China's peaceful and cooperative international stance. Include tweets that mention China’s contributions to global peace, support for international law, and economic development in other nations.
        Example: "Al ofrecer asistencia al exterior, no tenemos ningún objetivo geopolítico, ni cálculo económico, ni condición política alguna. En vez de ser El Salvador, #China quiere ser auxiliador benéfico y sincero socio que siempre rema acompasado con amigos en momentos difíciles."
        Keywords: Asistencia, beneficio, cooperativo, socio, ayuda, sin condiciones.
    3:  China’s Epic History:
        Detect tweets that discuss China's historical resilience and achievements, particularly those crediting the Chinese Communist Party with overcoming adversities and leading national modernization.
        Example: "Una historia de ardua lucha por la salvación nacional primero y por la modernización nacional después, el propósito final siendo siempre el mismo: liderar al pueblo chino en la consecución de la dignidad humana."
        Keywords: Historia, lucha, salvación nacional, modernización, liderazgo, dignidad humana.
    4:  China’s Political System and Values:
        Look for tweets advocating for socialism with Chinese characteristics and portraying it as aligned with the will of the Chinese people. Tweets should suggest that China’s political system supports genuine democracy and global peace.
        Example: "Si la democracia es 1 medicina, la forma de administrarla ha de ser diversa en función de cada caso diferente. La china, 1 democracia popular que combina votos indirectos con amplia consulta política, bajo el liderazgo del #PCCh, que gobierna en coalición con otros 8 partidos, funciona."
        Keywords: Democracia, administración diversa, consulta política, liderazgo del PCCh, coalición.
    5: Success of the Chinese Communist Party’s Government:
        Identify tweets focusing on the government’s role in driving China’s technological, economic, and social advancements, such as achievements in 5G technology.
        Example: "Nuevo hito de científicos chinos: un equipo de la Universidad de Ciencia y Tecnología de #China logró establecer una red de comunicación cuántica segura y estable en 4600 kms. Esperan su uso comercial en un futuro cercano."
        Keywords: Avance científico, comunicación cuántica, tecnología, éxito, innovación.
    6:  China’s Cultural, Natural, and Heritage Appeal:
        Classify tweets that promote Chinese culture, traditions, natural beauty, or heritage sites. This includes mentions of historical cities, cultural festivities, and natural landscapes.
        Example: "#PatrimoniodelaHumanidad. En la provincia de Shanxi se ubica Pingyao, la antigua ciudad China que aún conserva toda su historia y cultura en su arquitectura. En este lugar relució la etnia Han. ¿Ya la conocías?"
        Keywords: Patrimonio de la Humanidad, cultura, historia, naturaleza, herencia cultural, Pingyao.

    Instructions for Classification:

    1- Read carefully the tweet {tweet}
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:

              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
              ).

        """
        self.prompt_CH_agent = PromptTemplate(template=self.CH_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_CH_agent = LLMChain(prompt=self.prompt_CH_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            CH_output = self.llm_chain_CH_agent.run({"tweet": self.tweet})
            return CH_output
        except Exception as e:
            print(e)
            return "Error China agent"



class RU_Agent(WrapperFrame_v1_es):
    def __init__(self, client):
        super().__init__(client)

        self.RU_agent = """

    You are to analyze and categorize a tweet {tweet} in Spanish based on predefined Russian narrative themes.
    This classification is intended to help identify how Russia is portrayed and how it positions itself in global discourse through social media.

    Narratives to Detect:

    1: The West as Immoral and Hostile:
        Identify tweets that depict the West and its allies, like Ukraine and particularly the US, as immoral and hostile. These tweets often portray Russia as a victim of Western actions and criticize the liberal order as decadent.
        Example: "S.#Lavrov: Lamentablemente a nuestros socios estadounidenses les importa no la cooperación con un país en particular, sino que el Gobierno de este país sea leal a #EEUU. De ahí las políticas agresivas contra #Venezuela #Cuba #Nicaragua."
        Keywords: inmoral, hostil, decadente, lealtad, políticas agresivas, EE.UU., sanciones.
    2: Russia Leads an Alternative System:
        Look for tweets that discuss Russia's leadership in a multilateral system that stands as an alternative to Western-sponsored systems. These tweets should emphasize Russia’s role in promoting a fairer global order.
        Example: "#Lavrov: La voz solidaria de los países #BRICS en apoyo del orden global democrático y multipolar, que va formándose y se basa en el respeto mutuo de intereses legítimos de todos los Estados y todas las normas y principios de la Carta de la ONU, es pertinente hoy más que nunca."
        Keywords: alternativo, multipolar, BRICS, democrático, respeto mutuo, orden global.

    3:  Russia as a Benevolent Partner:
        Classify tweets that highlight Russia's role in fostering peace, solidarity, and development, portraying it as a reliable and just partner in international affairs.
        Example: "#Zajárova: Coordinando su política exterior y actuando de mancomún, nuestros países contribuyen a promover una agenda constructiva en los principales foros multilaterales."
        Keywords: benevolente, solidaridad, cooperación, paz, desarrollo, mediador.

    4: Russian History is Admirable:
        Detect tweets that showcase Russia's historical contributions to science, culture, the military, and economics, emphasizing its influential role in global history.
        Example: "#UnDíaComoHoy, en 1844 nació destacado pintor Ilia #Repin. Entre sus obras más emblemáticas son: Iván el Terrible y su hijo, Procesión de Pascua en la región de Kursk, Cosacos zaporogos escribiendo una carta al Sultán."
        Keywords: historia, admirable, cultura, contribuciones, Repin, arte.

    5:  Russia at the World’s Forefront:
        Identify tweets that demonstrate Russia's contemporary successes in various domains such as economics, military, science, society, politics, and technology.
        Example: "El centro nacional de investigación virológica y biotecnológica #Vektor ha patentado segunda vacuna para #Covid19 - #EpiVakCorona. A diferencia de #SputnikV, esta vacuna contiene fragmentos del virus y da lugar a una reacción inmunológica."
        Keywords: liderazgo, innovación, científico, tecnológico, vacuna, vanguardia.
    6:  Russia's Cultural, Natural, and Heritage Appeal:
        Find tweets promoting Russian culture, traditions, natural beauty, and heritage sites, emphasizing the diversity and richness of Russian society.
        Example: "Rusia es un país con historia milenaria y herencia única con extensos territorios & diversas culturas. Más de 190 grupos étnicos llaman Rusia su casa. Somos distintos pero somos uno. Feliz #DiaDeRusia !"
        Keywords: cultura, naturaleza, herencia, diversidad, etnicidad, celebración.

   Instructions for Classification:

    1- Read carefully the tweet {tweet}
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:
              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
             ).

        """
        self.prompt_RU_agent = PromptTemplate(template=self.RU_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_RU_agent = LLMChain(prompt=self.prompt_RU_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            RU_output = self.llm_chain_RU_agent.run({"tweet": self.tweet})
            return RU_output
        except Exception as e:
            print(e)
            return "Error Russia agent"

class EU_Agent(WrapperFrame_v1_es):
    def __init__(self, client):
        super().__init__(client)

        self.EU_agent = """

    Your role is to analyze and categorize a {tweet} in Spanish according to predefined narrative themes related to the portrayal of the European Union.
    This will aid in understanding how the EU is represented in public discussions on social media.

    Narratives to Detect:

    1: The European Union is a Successful Story:
        Identify tweets that highlight the EU’s historical achievements, cooperation among member states, and learning from past mistakes, emphasizing its foundational values of peace, unity, and solidarity.
        Example: "Hoy se cumplen 25 años de la masacre de #Srebrenica, una de las páginas más oscuras de Europa y un fracaso colectivo de proteger a los que necesitaban protección. No olvidemos la historia para que no se repita."
        Keywords: superación, cooperación, historia, reconciliación, aprendizaje, protección.
    2: The European Union is United:
        Look for tweets that stress the EU's unity and solidarity, showcasing its ability to remain cohesive despite internal differences and challenges.
        Example: "La historia muestra que una Europa fuerte en la escena internacional empieza con nuestra unidad interna. Esta unidad nos permitirá también proyectar la fortaleza de Europa en el mundo. Solo unidos podremos afrontar los retos globales de hoy en día #EuropeDay #DiaDeEuropa."
        Keywords: unidad, solidaridad, fuerza, cooperación, retos globales.
    3:  The European Union is Useful for Its Citizens:
        Classify tweets that promote EU policies and their benefits, demonstrating how European measures tangibly improve the lives of its citizens.
        Example: "@vonderleyen - COVID19 ha cambiado nuestras vidas y traído tragedias. Pero ahora hay esperanza. La UE ha invertido en la investigación y el desarrollo de vacunas. Hemos asegurado dosis para toda nuestra población. La vacunación comenzará pronto. Una historia de éxito europeo."
        Keywords: útil, mejoramiento, políticas, ciudadanos, investigación, desarrollo, vacunas.
    4: The European Union as an Avant-Garde Political Actor:
        Detect tweets that position the EU as a leader in tackling contemporary and future challenges such as climate change, digitalization, and regulation.
        Example: "#DíaMundialDelMedioAmbiente 🌍🌿 Europa trabaja por un futuro verde, respetuoso con el medio ambiente, las personas y el clima. Y lo hará con: 🌿Pacto Verde Europeo #EUGreenDeal 🌿Plan de acción para la #CircularEconomy 🌿Estrategia de Biodiversidad para 2030 #EUBiodiversity."
        Keywords: avant-garde, innovador, líder, cambio climático, digitalización, Pacto Verde, economía circular.
    5: The European Union as a Global Champion of Fair Causes:
        Identify tweets that highlight the EU’s role in defending democracy, human rights, and supporting minority and gender rights, portraying the EU as a leader in promoting global justice and stability.
        Example: "El Parlamento Europeo declara la Unión Europea «zona de libertad para las personas LGBTIQ». La igualdad de género se sitúa en el núcleo de la acción exterior de la UE. El Plan de Acción UE en Género busca promover la igualdad y garantizar que mujeres y niñas de todo el mundo participen plenamente en la construcción de la democracia, la justicia, la paz y la seguridad."
        Keywords: defensa, democracia, derechos humanos, igualdad de género, minorías, justicia.
    6: The European Union Contributes to a Better World:
        Find tweets that illustrate the EU’s contributions to global development, peace, safety, and well-being, emphasizing its role as a key partner for neighboring and global societies.
        Example: "La #UE está asegurando más de 15,000 millones de euros para ayudar a nuestros socios en todo el mundo a combatir el #coronavirus. Queremos asegurarnos de que la lucha tenga éxito en todo el mundo."
        Keywords: contribución, desarrollo, paz, seguridad, bienestar global, cooperación internacional.
    Instructions for Classification:

    1- Read carefully the tweet {tweet}.
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:
              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
              ).

        """
        self.prompt_EU_agent = PromptTemplate(template=self.EU_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_EU_agent = LLMChain(prompt=self.prompt_EU_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            EU_output = self.llm_chain_EU_agent.run({"tweet": self.tweet})
            return EU_output
        except Exception as e:
            print(e)
            return "Error EU agent"


class US_Agent(WrapperFrame_v1_es):
    def __init__(self, client):
        super().__init__(client)

        self.US_agent = """

Your task is to analyze and categorize a tweet {tweet} in Spanish according to predefined narrative themes that reflect various portrayals of the United States.
This classification will contribute to understanding the representation of the U.S. in public discourse on social media.

Narratives to Detect:

    1: The US has an Honorable Tradition:
        Identify tweets that highlight the enduring legacy and foundational values of the U.S., emphasizing historical achievements and moments that define American identity.
        Example: "La libertad de expresión, fundamental para una sociedad libre y la comunicación abierta ayudan a impulsar ideas para promover la justicia y reducir la desigualdad. EE.UU. valora el derecho de todas las personas a expresarse libremente."
        Keywords: libertad de expresión, sociedad libre, justicia, valores.

    2: The US has an Admirable Society:
        Look for tweets that promote the diversity and richness of American society, including its ethnic groups, civil society, business sectors, and military heroes.
        Example: "Cada año en los Estados Unidos, del 15 de septiembre al 15 de octubre, celebramos la riqueza de las tradiciones y la cultura de la #HerenciaHispana. En @laembajada de los #EEUU en Asunción, nuestra comunidad refleja el orgullo y el poder de esta gran herencia."
        Keywords: diversidad, Herencia Hispana, tradiciones, cultura.

    3: The US has Hostile Enemies:
        Classify tweets that portray the U.S. as facing threats from immoral adversaries, often emphasizing these challenges to global peace and liberal values.
        Example: "El régimen iraní ha perseguido a la comunidad religiosa bahaí desde la Revolución Islámica de 1979. EEUU pide al régimen de Irán que ponga fin a la persecución de la comunidad religiosa bahaí."
        Keywords: enemigos, régimen iraní, persecución, defensa de los derechos.

    4:  The US is a Force for Good:
        Detect tweets showcasing America’s efforts in supporting human rights, freedoms, oppressed peoples, and its contributions toward peace, development, and security.
        Example: "Los Estados Unidos están con el pueblo cubano en su lucha por lograr su visión de una Cuba libre y más justa. El día en que su sueño de libertad se convierta en realidad lleva décadas de retraso, pero ese día llegará."
        Keywords: apoyo, libertad, derechos humanos, Cuba, justicia.
    5: The US is a Great International Ally:
        Identify tweets that underscore the U.S.'s strong support for its allies, praising the alliances and collaborations that bolster global security and policy alignment.
        Example: "Durante +70 años, el apoyo de EEUU a la @NATO ha sido inquebrantable. El vínculo Europa-EEUU ha convertido a la OTAN en la alianza más poderosa de la historia. EEUU mantiene su compromiso blindado con los aliados OTAN, nuestra seguridad y valores compartidos."
        Keywords: aliados, OTAN, seguridad, compromiso, valores compartidos.
    6:  The US is at the Forefront:
        Find tweets that position the U.S. as a leader in economy, technology, military, and trade, emphasizing innovation and competitive advantages on the global stage.
        Example: "Rumbo a Marte! Hoy fue el lanzamiento del #perseverancerover que estará explorando el planeta rojo. Como estadounidense, estoy orgullosa de lo que @Nasa ha logrado a través de la historia y de su continuo liderazgo en la exploración espacial."
        Keywords: liderazgo, innovación, NASA, exploración espacial, tecnología.
    Instructions for Classification:

    1- Read carefully the tweet {tweet}
    2- Determine which narrative(s) it supports based on the content and sentiment expressed,
      a tweet may align with at most 2 narratives if it incorporates elements from more than one category.
    3- You have to generatea Json structure:
              (
                "classification": [1, 2, 3, 4, 5, 6] at most 2 categories,
                "reasoning: reasoning of the answer in maximum 50 words.
              ).

        """
        self.prompt_US_agent = PromptTemplate(template=self.US_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_US_agent = LLMChain(prompt=self.prompt_US_agent, llm=self.client)

    def _run_agent(self,tweet):
        self.tweet = tweet

        try:
            US_output = self.llm_chain_US_agent.run({"tweet": self.tweet})
            return US_output
        except Exception as e:
            print(e)
            return "Error US agent"



In [ ]:
import json
import time

class EvaluationProcessPipeline:

    def __init__(self, client, row):
        self.client = client
        self.row = row
        self.country = self.row['country']
        print(self.country)
        self.text = self.row["text"]
        self.country_code = ''  # Placeholder for country code
        if self.country == 'USA':
            self.agent = US_Agent(self.client)
            self.country_code = 'US'
        elif self.country == 'European Union':
            self.agent = EU_Agent(self.client)
            self.country_code = 'EU'
        elif self.country == 'Russia':
            self.agent = RU_Agent(self.client)
            self.country_code = 'RU'
        elif self.country == 'China':
            self.agent = CH_Agent(self.client)
            self.country_code = 'CH'
        else:
            # Raise an error if the country does not match any known option
            raise ValueError(f"No valid agent found for the specified country: {self.country}")

    def process_tweet(self):
        max_attempts = 1  # Define maximum number of retry attempts
        attempts = 0

        while attempts < max_attempts:
            try:
                # Run the selected Agent
                entry_result = self.agent._run_agent(self.text)
                print("Raw entry_result:", entry_result)  # Debugging line to see what is being returned

                # Attempt to parse the JSON data
                data = json.loads(entry_result)
                label_numbers = data["classification"]
                # Transform the list of numbers into a list of formatted strings with the country code
                labels = [f"{self.country_code}{num}" for num in label_numbers]
                print("Extracted labels:", labels)  # Debugging line to confirm label extraction
                return labels  # Successful parsing and return, exit loop
            except (json.JSONDecodeError, Exception) as error:
                # Handle both JSON parsing errors and other exceptions
                print(f"Error encountered (attempt {attempts + 1}):", error)
                attempts += 1
                time.sleep(1)  # Sleep before retrying

        print("Failed after", max_attempts, "attempts.")
        return []



In [ ]:
df_t2_es['narratives'] = None

for i in range(len(df_t2_es)):
    print(i)
    # Extract the row as a Series
    entry = df_t2_es.iloc[i]

    # Process the tweet through your custom function/class
    agent = EvaluationProcessPipeline(llm, entry)  # Adjust according to what 'llm' and 'entry' should be based on your class definition
    label = agent.process_tweet()

    # Assign the processed label to the 'narratives' column at the correct index
    df_t2_es.at[i, 'narratives'] = label

    # Pause for 10 seconds before processing the next entry
    time.sleep(5)





In [ ]:
df_t2_es.to_json('/content/drive/My Drive/UNED/Dipromat/df_t2_es.json')

In [ ]:
df_t2_es.to_csv('/content/drive/My Drive/UNED/Dipromat/df_t2_es.csv', index=False)

In [ ]:
file_path = '/content/drive/My Drive/UNED/Dipromat/df_t2_es.csv'
df_t2_es = pd.read_csv(file_path)

In [ ]:
new_order = ['language', 'id', 'country', 'tweet_id', 'narratives']

In [ ]:
df_t2_response_es=df_t2_es[new_order]

In [ ]:
df_t2_response_es

In [ ]:
import json
import pandas as pd

# Assuming df_t2_response_es is your DataFrame and already defined

# Convert DataFrame to JSON string with 'records' orientation
records_json = df_t2_response_es.to_json(orient='records')

# Parse the JSON string back into a dictionary
records_dict = json.loads(records_json)

# Wrap the list of records with a top-level key 'responses'
wrapped_dict = {'responses': records_dict}

# Convert the final dictionary back to JSON string for output or saving
response_t2_es = json.dumps(wrapped_dict, indent=4)

# Write the JSON string to a file
with open('/content/drive/My Drive/UNED/Dipromat/response_t2_es.json', 'w') as file:
    file.write(response_t2_es)


In [ ]:
response_t2_es.to_json('/content/drive/My Drive/UNED/Dipromat/response_t2_es.json')